# Position MC Report

This notebook calculates Marginal Contribution (MC) to total portfolio risk broken down by position/strategy from `pos_summary.csv`, with separate sections for single-product analysis and cross-product aggregation.

In [64]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


## 1. Configuration & Data Loading


In [65]:
# ============================================================================
# CONFIGURATION - Reuse from q_risk_report.ipynb
# ============================================================================

# Bucket definitions
front = ["A01", "A02", "A03", "A04"]
mid   = ["A05", "A06", "A07", "A08"]
back  = ["A09", "A10", "A11", "A12", "A13", "A14", "A15"]

# EWMA decay parameters (lambda)
lambda_front = 0.97
lambda_mid   = 0.98
lambda_back  = 0.99

# EWMA initialization
ewma_init_obs = 60  # Use first N observations for sample covariance initialization

# Data file paths
data_file = "data_.csv"
delta_summary_file = "delta_summary.csv"
delta_positions_file = "delta_positions.csv"
pos_summary_file = "pos_summary.csv"
product_mapping_file = "product_mapping.csv"
holidays_file = "holidays.csv"

# Load all data files
print("Loading data files...")
pos_summary_df = pd.read_csv(pos_summary_file)
delta_positions_df = pd.read_csv(delta_positions_file)
delta_summary_df = pd.read_csv(delta_summary_file)
df_raw = pd.read_csv(data_file)
product_mapping_df = pd.read_csv(product_mapping_file)

# Load holidays and create set for fast lookup
holidays_df = pd.read_csv(holidays_file, header=None, names=['date'])
holidays_df = holidays_df.dropna()  # Remove empty rows
holidays_df['date'] = pd.to_datetime(holidays_df['date'], format='%m/%d/%Y', errors='coerce')
holidays_df = holidays_df.dropna()  # Remove any rows that couldn't be parsed
holiday_dates = set(holidays_df['date'].dt.date)  # Use .date() for date-only comparison

# Parse date column for price data
df_raw['date'] = pd.to_datetime(df_raw['date'])
df_raw = df_raw.set_index('date').sort_index()

# Build contract-to-node mapping (same as q_risk_report.ipynb)
# Map tenors chronologically to A01-A15 (first 15 tenors in delta_summary.csv)
unique_tenors = delta_summary_df['Tenor'].unique()
contract_to_node = {}
for idx, tenor in enumerate(unique_tenors[:15]):  # Only map first 15 to A01-A15
    node_code = f"A{idx+1:02d}"
    contract_to_node[tenor] = node_code

# All nodes
all_nodes = [f"A{i:02d}" for i in range(1, 16)]
n_total = len(all_nodes)

# Build product mapping dictionary from CSV
product_to_mapped = dict(zip(product_mapping_df['Product'], product_mapping_df['Mapping']))

# Build mapping from Mapped_Product to lowercase data column prefix
# This maps mapped products (HTT, HOUBR, etc.) to their lowercase data column names (htt, houbr, etc.)
mapped_to_data_column = {
    'HTT': 'htt',
    'HOUBR': 'houbr',
    'CLBR': 'clbr',
    'WDF': 'wdf',
    'LH': 'lh'  # Longhorn
}

print(f"Loaded {len(pos_summary_df)} positions from pos_summary.csv")
print(f"Loaded {len(delta_positions_df)} expanded positions from delta_positions.csv")
print(f"Loaded {len(delta_summary_df)} tenors from delta_summary.csv")
print(f"Loaded {len(product_mapping_df)} product mappings from product_mapping.csv")
print(f"Loaded {len(holiday_dates)} holiday dates from holidays.csv")
print(f"Contract-to-node mappings: {len(contract_to_node)} contracts")
print(f"\nProduct mapping (Product -> Mapped_Product):")
for orig, mapped in product_to_mapped.items():
    print(f"  {orig} -> {mapped}")
print(f"\nUnique strategies: {sorted(pos_summary_df['Strategy'].unique())}")
print(f"Unique products: {sorted(pos_summary_df['Product'].unique())}")


Loading data files...
Loaded 20 positions from pos_summary.csv
Loaded 100 expanded positions from delta_positions.csv
Loaded 34 tenors from delta_summary.csv
Loaded 9 product mappings from product_mapping.csv
Loaded 111 holiday dates from holidays.csv
Contract-to-node mappings: 15 contracts

Product mapping (Product -> Mapped_Product):
  HTT -> HTT
  HOUBR_Cross -> HOUBR
  HOUBR -> HOUBR
  CLBR -> CLBR
  HTT Rolls -> HTT
  HOUBR Boxes -> HOUBR
  CLBR Boxes -> CLBR
  HTTMID -> LH
  WDF -> WDF

Unique strategies: ['Freight', 'HOUBR_Back', 'HOUBR_Front', 'HOUBR_rolls', 'HTT_Back', 'HTT_Front', 'HTT_Mid', 'HTT_Rolls', 'Longhorn']
Unique products: ['CLBR', 'CLBR Boxes', 'HOUBR', 'HOUBR Boxes', 'HOUBR_Cross', 'HTT', 'HTT Rolls', 'HTTMID', 'WDF']


## 2. Reuse Functions from q_risk_report.ipynb


In [66]:
def compute_ewma_covariance(returns_df, nodes, product, lambda_val, init_obs=60):
    """
    Compute EWMA covariance matrix for given nodes.
    
    Parameters:
    -----------
    returns_df : DataFrame
        DataFrame with returns (daily changes)
    nodes : list
        List of node names (e.g., ['A01', 'A02', ...])
    product : str
        Product name (e.g., 'htt', 'houbr', etc.)
    lambda_val : float
        EWMA decay parameter
    init_obs : int
        Number of observations to use for initial covariance
    
    Returns:
    --------
    cov_matrix : ndarray
        Final EWMA covariance matrix
    """
    # Extract relevant columns
    cols = [f'{product}_{node}' for node in nodes]
    returns_subset = returns_df[cols].values
    
    n_obs, n_nodes = returns_subset.shape
    
    if n_obs < init_obs:
        raise ValueError(f"Need at least {init_obs} observations, got {n_obs}")
    
    # Initialize with sample covariance of first N observations
    init_returns = returns_subset[:init_obs]
    # Remove any rows with NaN
    init_returns = init_returns[~np.isnan(init_returns).any(axis=1)]
    
    if len(init_returns) < 10:
        # Fallback: use identity matrix scaled by variance
        cov_current = np.eye(n_nodes) * np.var(returns_subset, axis=0).mean()
    else:
        cov_current = np.cov(init_returns.T)
    
    # EWMA recursion: Σ_t = λ * Σ_{t-1} + (1-λ) * r_t * r_t'
    for t in range(init_obs, n_obs):
        r_t = returns_subset[t:t+1, :]  # Shape: (1, n_nodes)
        
        # Skip if any NaN
        if np.isnan(r_t).any():
            continue
        
        # EWMA update
        outer_product = np.outer(r_t, r_t)
        cov_current = lambda_val * cov_current + (1 - lambda_val) * outer_product[0]
    
    return cov_current

def compute_mc_to_total(w_position, w_total, Sigma_total):
    """
    Compute marginal contribution of a position to total portfolio.
    MC = 1000 * (w_position' Σ_total w_total) / sqrt(w_total' Σ_total w_total)
    
    Parameters:
    -----------
    w_position : ndarray
        Position vector (15 nodes)
    w_total : ndarray
        Total portfolio vector (15 nodes)
    Sigma_total : ndarray
        Total covariance matrix (15x15)
    
    Returns:
    --------
    mc : float
        Marginal contribution to total portfolio risk
    """
    # Compute numerator: w_position' Σ_total w_total
    numerator = w_position.T @ Sigma_total @ w_total
    
    # Compute denominator: sqrt(w_total' Σ_total w_total)
    total_var = w_total.T @ Sigma_total @ w_total
    if total_var <= 0:
        return 0.0
    denominator = np.sqrt(total_var)
    
    return 1000 * numerator / denominator

print("Functions loaded: compute_ewma_covariance, compute_mc_to_total")


Functions loaded: compute_ewma_covariance, compute_mc_to_total


## 3. Strategy Position Mapping


In [67]:
def build_strategy_position_vector(strategy_name, product, delta_positions_df, contract_to_node, all_nodes):
    """
    Build position vector for a strategy within a product.
    
    Parameters:
    -----------
    strategy_name : str
        Strategy name (e.g., 'HTT_Front')
    product : str
        Mapped product name (e.g., 'HTT', 'HOUBR', etc.)
    delta_positions_df : DataFrame
        Expanded positions with Strategy and Mapped_Product columns
    contract_to_node : dict
        Mapping from tenor to node code
    all_nodes : list
        List of all node codes (A01-A15)
    
    Returns:
    --------
    w_strategy : ndarray
        Position vector (15 nodes) for this strategy
    """
    # Filter delta_positions for this strategy and product
    strategy_positions = delta_positions_df[
        (delta_positions_df['Strategy'] == strategy_name) & 
        (delta_positions_df['Mapped_Product'] == product)
    ].copy()
    
    # Initialize position vector
    w_strategy = np.zeros(len(all_nodes))
    
    # Map each tenor to node and accumulate quantities
    for _, row in strategy_positions.iterrows():
        tenor = row['Tenor']
        qty = row['Qty']
        
        if tenor in contract_to_node:
            node = contract_to_node[tenor]
            node_idx = all_nodes.index(node)
            w_strategy[node_idx] += qty
    
    return w_strategy

def determine_bucket(w_position, front, mid, back, all_nodes):
    """
    Determine which bucket(s) a position belongs to based on non-zero nodes.
    
    Returns:
    --------
    bucket : str
        'Front', 'Mid', 'Back', 'Mixed', or 'None'
    """
    front_positions = w_position[[all_nodes.index(n) for n in front]]
    mid_positions = w_position[[all_nodes.index(n) for n in mid]]
    back_positions = w_position[[all_nodes.index(n) for n in back]]
    
    has_front = np.any(np.abs(front_positions) > 1e-10)
    has_mid = np.any(np.abs(mid_positions) > 1e-10)
    has_back = np.any(np.abs(back_positions) > 1e-10)
    
    buckets = []
    if has_front:
        buckets.append('Front')
    if has_mid:
        buckets.append('Mid')
    if has_back:
        buckets.append('Back')
    
    if len(buckets) == 0:
        return 'None'
    elif len(buckets) == 1:
        return buckets[0]
    else:
        return 'Mixed'

print("Strategy mapping functions loaded")


Strategy mapping functions loaded


## 4. Single Product MC Calculation


In [68]:
# Get all unique mapped products from delta_positions
all_products = sorted(delta_positions_df['Mapped_Product'].unique())

print(f"Products to process: {all_products}")
print(f"Mapped Product -> Data Column mapping: {mapped_to_data_column}")

# Store results
position_mc_results = []
diagnostic_data = []  # Store diagnostic information

# Process each product
for mapped_product in all_products:
    product_lower = mapped_to_data_column.get(mapped_product, mapped_product.lower())
    
    print(f"\n{'='*80}")
    print(f"Processing product: {mapped_product} (data column: {product_lower})")
    print(f"{'='*80}")
    
    # Check if product data exists
    product_cols = [col for col in df_raw.columns if col.startswith(f'{product_lower}_A') and '/' not in col]
    if not product_cols:
        print(f"  WARNING: No data columns found for {product_lower}, skipping...")
        continue
    
    # Extract price data for this product
    product_cols = sorted(product_cols, key=lambda x: int(x.split('_A')[1]))
    df_prices = df_raw[product_cols].copy()
    
    # Compute daily changes (returns)
    df_returns = df_prices.diff().dropna()
    df_returns = df_returns.dropna()
    
    # Filter out holiday dates
    initial_count = len(df_returns)
    # Convert index dates to a list and check membership
    is_holiday = [date.date() in holiday_dates for date in df_returns.index]
    df_returns = df_returns[~pd.Series(is_holiday, index=df_returns.index)]
    removed_count = initial_count - len(df_returns)
    
    if len(df_returns) < ewma_init_obs:
        print(f"  WARNING: Insufficient data ({len(df_returns)} < {ewma_init_obs}), skipping...")
        continue
    
    print(f"  Returns data shape: {df_returns.shape}")
    print(f"  Removed {removed_count} holiday dates from returns data (out of {initial_count} total)")
    
    # Compute EWMA covariance for each bucket
    print("  Computing EWMA covariances...")
    Sigma_front = compute_ewma_covariance(df_returns, front, product_lower, lambda_front, ewma_init_obs)
    Sigma_mid = compute_ewma_covariance(df_returns, mid, product_lower, lambda_mid, ewma_init_obs)
    Sigma_back = compute_ewma_covariance(df_returns, back, product_lower, lambda_back, ewma_init_obs)
    
    # Build total covariance matrix (block-diagonal)
    n_front = len(front)
    n_mid = len(mid)
    n_back = len(back)
    
    Sigma_total = np.zeros((n_total, n_total))
    Sigma_total[:n_front, :n_front] = Sigma_front
    Sigma_total[n_front:n_front+n_mid, n_front:n_front+n_mid] = Sigma_mid
    Sigma_total[n_front+n_mid:, n_front+n_mid:] = Sigma_back
    
    print(f"  Total covariance shape: {Sigma_total.shape}")
    
    # Build total portfolio position vector for this product from delta_summary
    w_total = np.zeros(n_total)
    if mapped_product in delta_summary_df.columns:
        for _, row in delta_summary_df.iterrows():
            tenor = row['Tenor']
            position = row[mapped_product]
            if abs(position) > 1e-10 and tenor in contract_to_node:
                node = contract_to_node[tenor]
                node_idx = all_nodes.index(node)
                w_total[node_idx] = float(position)
    
    total_qty = np.sum(np.abs(w_total))
    print(f"  Total portfolio quantity: {total_qty:.2f}")
    
    # Display position vector
    print(f"  Position vector (nodes A01-A15):")
    for i, node in enumerate(all_nodes):
        if abs(w_total[i]) > 1e-10:
            print(f"    {node}: {w_total[i]:.2f}")
    
    # Get all strategies for this product
    product_strategies = delta_positions_df[
        delta_positions_df['Mapped_Product'] == mapped_product
    ]['Strategy'].unique()
    
    print(f"  Found {len(product_strategies)} strategies: {sorted(product_strategies)}")
    
    # Calculate MC for each strategy
    for strategy_name in sorted(product_strategies):
        # Build strategy position vector
        w_strategy = build_strategy_position_vector(
            strategy_name, mapped_product, delta_positions_df, 
            contract_to_node, all_nodes
        )
        
        strategy_qty = np.sum(np.abs(w_strategy))
        if strategy_qty < 1e-10:
            continue  # Skip empty strategies
        
        # Calculate MC
        mc = compute_mc_to_total(w_strategy, w_total, Sigma_total)
        
        # Determine bucket
        bucket = determine_bucket(w_strategy, front, mid, back, all_nodes)
        
        # Count unique tenors for this strategy
        strategy_tenors = delta_positions_df[
            (delta_positions_df['Strategy'] == strategy_name) & 
            (delta_positions_df['Mapped_Product'] == mapped_product)
        ]['Tenor'].nunique()
        
        # Store result
        position_mc_results.append({
            'Strategy': strategy_name,
            'Product': mapped_product,
            'MC_to_total': mc,
            'Qty_total': strategy_qty,
            'Tenor_count': strategy_tenors,
            'Bucket': bucket
        })
        
        print(f"    {strategy_name}: MC={mc:.2f}, Qty={strategy_qty:.2f}, Bucket={bucket}")

print(f"\n{'='*80}")
print(f"Completed processing all products")
print(f"Total strategy/product combinations: {len(position_mc_results)}")
print(f"{'='*80}")


Products to process: ['CLBR', 'HOUBR', 'HTT', 'LH', 'WDF']
Mapped Product -> Data Column mapping: {'HTT': 'htt', 'HOUBR': 'houbr', 'CLBR': 'clbr', 'WDF': 'wdf', 'LH': 'lh'}

Processing product: CLBR (data column: clbr)
  Returns data shape: (968, 15)
  Removed 10 holiday dates from returns data (out of 978 total)
  Computing EWMA covariances...


  Total covariance shape: (15, 15)
  Total portfolio quantity: 2368.00
  Position vector (nodes A01-A15):
    A02: -268.00
    A05: 300.00
    A07: 850.00
    A08: -850.00
    A10: 100.00
  Found 2 strategies: ['HOUBR_Back', 'HOUBR_rolls']
    HOUBR_Back: MC=-2186.87, Qty=32.00, Bucket=Front
    HOUBR_rolls: MC=35086.98, Qty=2400.00, Bucket=Mixed

Processing product: HOUBR (data column: houbr)
  Returns data shape: (964, 15)
  Removed 8 holiday dates from returns data (out of 972 total)
  Computing EWMA covariances...
  Total covariance shape: (15, 15)
  Total portfolio quantity: 3925.00
  Position vector (nodes A01-A15):
    A01: 300.00
    A02: -300.00
    A03: -1300.00
    A04: 1300.00
    A08: 200.00
    A09: 200.00
    A10: 200.00
    A11: -25.00
    A12: -25.00
    A13: -25.00
    A14: -25.00
    A15: -25.00
  Found 3 strategies: ['HOUBR_Back', 'HOUBR_Front', 'HOUBR_rolls']
    HOUBR_Back: MC=21552.57, Qty=725.00, Bucket=Mixed
    HOUBR_Front: MC=0.00, Qty=600.00, Bucket=Front
  

## 5. Results DataFrame


In [69]:
# Check if diagnostic_data exists (from running cell 9)
if 'diagnostic_data' not in globals() or len(diagnostic_data) == 0:
    print("WARNING: diagnostic_data not found. Please run cell 9 (Single Product MC Calculation) first.")
    print("Skipping diagnostics...")
    diagnostic_df = pd.DataFrame()
else:
    # Create diagnostic DataFrame
    diagnostic_df = pd.DataFrame(diagnostic_data)
    
    print("="*80)
    print("MC CALCULATION DIAGNOSTICS")
    print("="*80)
    
    # Display diagnostic breakdown for each strategy
    print("\nDetailed MC Calculation Components:")
    print("-"*80)
    for _, row in diagnostic_df.iterrows():
        print(f"\n{row['Strategy']} ({row['Product']}):")
        print(f"  Strategy Q (standalone): {row['Strategy_Q_standalone']:.2f}")
        print(f"  Total Portfolio Q: {row['Total_Q_portfolio']:.2f}")
        print(f"  MC Numerator (w_strategy' Σ w_total): {row['MC_numerator']:.2f}")
        print(f"  MC Denominator (sqrt(w_total' Σ w_total)): {row['MC_denominator']:.2f}")
        print(f"  MC Ratio (numerator/denominator): {row['MC_ratio']:.2f}")
        print(f"  Final MC (1000 * ratio): {row['MC_to_total']:.2f}")
        print(f"  Strategy Variance: {row['Strategy_var']:.6f}")
        print(f"  Total Portfolio Variance: {row['Total_var']:.6f}")
    
    # Compare high MC strategies
    print("\n" + "="*80)
    print("COMPARISON: High MC Strategies vs HTT")
    print("="*80)
    high_mc_strategies = diagnostic_df.nlargest(3, 'MC_to_total', keep='all')
    htt_strategies = diagnostic_df[diagnostic_df['Product'] == 'HTT'].nlargest(3, 'MC_to_total', keep='all')
    
    print("\nTop 3 Highest MC Strategies:")
    print(high_mc_strategies[['Strategy', 'Product', 'MC_to_total', 'Strategy_Q_standalone', 
                              'Total_Q_portfolio', 'MC_ratio', 'Qty_total']].to_string(index=False))
    
    print("\nTop 3 HTT Strategies (for comparison):")
    print(htt_strategies[['Strategy', 'Product', 'MC_to_total', 'Strategy_Q_standalone', 
                          'Total_Q_portfolio', 'MC_ratio', 'Qty_total']].to_string(index=False))


Skipping diagnostics...


In [70]:
# Create position MC DataFrame
position_mc_df = pd.DataFrame(position_mc_results)

# Add MC sign indicator
position_mc_df['MC_signed'] = position_mc_df['MC_to_total'].apply(
    lambda x: 'POS' if x > 0 else 'NEG' if x < 0 else 'ZERO'
)

# Sort by absolute MC (descending)
position_mc_df['abs_MC'] = position_mc_df['MC_to_total'].abs()
position_mc_df = position_mc_df.sort_values('abs_MC', ascending=False).reset_index(drop=True)
position_mc_df = position_mc_df.drop('abs_MC', axis=1)

print("Position MC DataFrame:")
print(position_mc_df.to_string(index=False))
print(f"\nTotal rows: {len(position_mc_df)}")

# Build and display position vectors summary for all products
print("\n" + "="*80)
print("POSITION VECTORS BY PRODUCT (w_total for each product)")
print("="*80)

# Rebuild position vectors for display
product_position_vectors = {}
for mapped_product in sorted(delta_positions_df['Mapped_Product'].unique()):
    w_total = np.zeros(n_total)
    if mapped_product in delta_summary_df.columns:
        for _, row in delta_summary_df.iterrows():
            tenor = row['Tenor']
            position = row[mapped_product]
            if abs(position) > 1e-10 and tenor in contract_to_node:
                node = contract_to_node[tenor]
                node_idx = all_nodes.index(node)
                w_total[node_idx] = float(position)
    
    # Only show if there are positions
    total_qty = np.sum(np.abs(w_total))
    if total_qty > 1e-10:
        product_position_vectors[mapped_product] = w_total.copy()
        print(f"\n{mapped_product} Position Vector (Total Qty: {total_qty:.2f}):")
        print(f"  Node positions:")
        for i, node in enumerate(all_nodes):
            if abs(w_total[i]) > 1e-10:
                print(f"    {node}: {w_total[i]:.2f}")
        
        # Also show as a compact vector
        non_zero_nodes = [f"{node}={w_total[i]:.2f}" for i, node in enumerate(all_nodes) if abs(w_total[i]) > 1e-10]
        print(f"  Compact: {', '.join(non_zero_nodes)}")

# Create a DataFrame showing position vectors as a table
if product_position_vectors:
    position_vectors_df = pd.DataFrame(product_position_vectors, index=all_nodes).T
    # Only show non-zero columns
    non_zero_cols = [col for col in position_vectors_df.columns if position_vectors_df[col].abs().sum() > 1e-10]
    position_vectors_df = position_vectors_df[non_zero_cols]
    
    print("\n" + "="*80)
    print("POSITION VECTORS TABLE (Products × Nodes)")
    print("="*80)
    print(position_vectors_df.to_string())


Position MC DataFrame:
   Strategy Product   MC_to_total  Qty_total  Tenor_count Bucket MC_signed
  HTT_Rolls     HTT  1.765134e+05  8900.0000           30  Mixed       POS
HOUBR_rolls    CLBR  3.508698e+04  2400.0000            6  Mixed       POS
 HOUBR_Back   HOUBR  2.155257e+04   725.0000           15  Mixed       POS
    HTT_Mid     HTT  1.641687e+04   574.0000            1  Front       POS
   HTT_Back     HTT  5.510005e+03   975.0000           18  Mixed       POS
  HTT_Front     HTT -2.860082e+03   100.0000            1  Front       NEG
 HOUBR_Back    CLBR -2.186870e+03    31.9996            1  Front       NEG
HOUBR_rolls   HOUBR  3.012809e-07  2600.0000            2  Front       POS
HOUBR_Front   HOUBR  8.700723e-08   600.0000            2  Front       POS
    Freight     WDF  0.000000e+00   145.0000            3  Front      ZERO

Total rows: 10

POSITION VECTORS BY PRODUCT (w_total for each product)

CLBR Position Vector (Total Qty: 2368.00):
  Node positions:
    A02: -268.00
 

## 6. Aggregation by Strategy and Product


In [71]:
# Aggregate by Strategy (across all products)
mc_by_strategy = position_mc_df.groupby('Strategy').agg({
    'MC_to_total': 'sum',
    'Qty_total': 'sum',
    'Tenor_count': 'sum',
    'Product': lambda x: ', '.join(sorted(x.unique()))
}).reset_index()
mc_by_strategy.columns = ['Strategy', 'MC_to_total', 'Qty_total', 'Tenor_count', 'Products']
mc_by_strategy['MC_signed'] = mc_by_strategy['MC_to_total'].apply(
    lambda x: 'POS' if x > 0 else 'NEG' if x < 0 else 'ZERO'
)
mc_by_strategy = mc_by_strategy.sort_values('MC_to_total', key=abs, ascending=False).reset_index(drop=True)

print("="*80)
print("MC AGGREGATION BY STRATEGY (across all products)")
print("="*80)
print(mc_by_strategy.to_string(index=False))

# Aggregate by Product (across all strategies)
mc_by_product = position_mc_df.groupby('Product').agg({
    'MC_to_total': 'sum',
    'Qty_total': 'sum',
    'Tenor_count': 'sum',
    'Strategy': lambda x: len(x.unique())
}).reset_index()
mc_by_product.columns = ['Product', 'MC_to_total', 'Qty_total', 'Tenor_count', 'Strategy_count']
mc_by_product['MC_signed'] = mc_by_product['MC_to_total'].apply(
    lambda x: 'POS' if x > 0 else 'NEG' if x < 0 else 'ZERO'
)
mc_by_product = mc_by_product.sort_values('MC_to_total', key=abs, ascending=False).reset_index(drop=True)

print("\n" + "="*80)
print("MC AGGREGATION BY PRODUCT (across all strategies)")
print("="*80)
print(mc_by_product.to_string(index=False))


MC AGGREGATION BY STRATEGY (across all products)
   Strategy   MC_to_total  Qty_total  Tenor_count    Products MC_signed
  HTT_Rolls  1.765134e+05  8900.0000           30         HTT       POS
HOUBR_rolls  3.508698e+04  5000.0000            8 CLBR, HOUBR       POS
 HOUBR_Back  1.936570e+04   756.9996           16 CLBR, HOUBR       POS
    HTT_Mid  1.641687e+04   574.0000            1         HTT       POS
   HTT_Back  5.510005e+03   975.0000           18         HTT       POS
  HTT_Front -2.860082e+03   100.0000            1         HTT       NEG
HOUBR_Front  8.700723e-08   600.0000            2       HOUBR       POS
    Freight  0.000000e+00   145.0000            3         WDF      ZERO

MC AGGREGATION BY PRODUCT (across all strategies)
Product   MC_to_total  Qty_total  Tenor_count  Strategy_count MC_signed
    HTT 195580.158615 10549.0000           50               4       POS
   CLBR  32900.115037  2431.9996            7               2       POS
  HOUBR  21552.570271  3925.0000    

## 7. Cross-Product MC Summary


## 9. Portfolio-Wide Hedge Recommender

This section recommends hedge instruments from HTT or CLBR to reduce total portfolio risk using the shared covariance matrix. Unlike the single-product hedge recommender in `q_risk_report.ipynb`, this evaluates hedges at the portfolio level, accounting for cross-product correlations.

### 9.1 Helper Functions

In [72]:
# Cross-product summary
# Note: This assumes independence between products (sums MCs)
# For full cross-product correlation, would need multi-product covariance matrix

total_mc_all_products = position_mc_df['MC_to_total'].sum()
total_qty_all_products = position_mc_df['Qty_total'].sum()

print("="*80)
print("CROSS-PRODUCT MC SUMMARY")
print("="*80)
print(f"Total MC across all products (sum of single-product MCs): {total_mc_all_products:.2f}")
print(f"Total quantity across all products: {total_qty_all_products:.2f}")
print(f"\nNote: This assumes independence between products.")
print(f"      For full cross-product correlation, a multi-product covariance matrix would be needed.")

# Strategy × Product pivot table
strategy_product_pivot = position_mc_df.pivot_table(
    index='Strategy',
    columns='Product',
    values='MC_to_total',
    aggfunc='sum',
    fill_value=0.0
)

print("\n" + "="*80)
print("MC BY STRATEGY × PRODUCT (Pivot Table)")
print("="*80)
print(strategy_product_pivot.to_string())

# Top contributors
print("\n" + "="*80)
print("TOP 10 CONTRIBUTORS BY ABSOLUTE MC")
print("="*80)
top_contributors = position_mc_df.head(10)[['Strategy', 'Product', 'MC_to_total', 'Qty_total', 'Bucket']]
print(top_contributors.to_string(index=False))


CROSS-PRODUCT MC SUMMARY
Total MC across all products (sum of single-product MCs): 250032.84
Total quantity across all products: 17051.00

Note: This assumes independence between products.
      For full cross-product correlation, a multi-product covariance matrix would be needed.

MC BY STRATEGY × PRODUCT (Pivot Table)
Product              CLBR         HOUBR            HTT  WDF
Strategy                                                   
Freight          0.000000  0.000000e+00       0.000000  0.0
HOUBR_Back   -2186.869610  2.155257e+04       0.000000  0.0
HOUBR_Front      0.000000  8.700723e-08       0.000000  0.0
HOUBR_rolls  35086.984647  3.012809e-07       0.000000  0.0
HTT_Back         0.000000  0.000000e+00    5510.005152  0.0
HTT_Front        0.000000  0.000000e+00   -2860.081791  0.0
HTT_Mid          0.000000  0.000000e+00   16416.869483  0.0
HTT_Rolls        0.000000  0.000000e+00  176513.365772  0.0

TOP 10 CONTRIBUTORS BY ABSOLUTE MC
   Strategy Product   MC_to_total  Qty_tot

In [73]:
def build_hedge_vector(hedge_instrument, hedge_product, product_indices, all_nodes, n_combined):
    """
    Convert a hedge instrument (node or spread) into a position vector in the combined space.
    
    Parameters:
    -----------
    hedge_instrument : str
        Instrument name, either outright (e.g., 'A01') or spread (e.g., 'A01/A02')
    hedge_product : str
        Product name ('HTT' or 'CLBR')
    product_indices : dict
        Dictionary mapping product -> (start_idx, end_idx) in combined space
    all_nodes : list
        List of all node codes ['A01', ..., 'A15']
    n_combined : int
        Total length of combined vector
    
    Returns:
    --------
    h_hedge : ndarray
        Position vector of length n_combined with hedge position
    """
    # Initialize zero vector
    h_hedge = np.zeros(n_combined)
    
    # Check if hedge product exists in product_indices
    if hedge_product not in product_indices:
        return h_hedge
    
    # Get product's slice in combined space
    i_start, i_end = product_indices[hedge_product]
    
    # Check if it's a spread (contains '/')
    if '/' in hedge_instrument:
        # Spread: e.g., 'A01/A02' means long A01, short A02
        parts = hedge_instrument.split('/')
        if len(parts) != 2:
            return h_hedge  # Invalid spread format
        
        node1, node2 = parts[0], parts[1]
        
        # Find node indices
        if node1 in all_nodes and node2 in all_nodes:
            idx1 = all_nodes.index(node1)
            idx2 = all_nodes.index(node2)
            
            # Long first node, short second node
            h_hedge[i_start + idx1] = 1.0
            h_hedge[i_start + idx2] = -1.0
    else:
        # Outright: e.g., 'A01'
        if hedge_instrument in all_nodes:
            node_idx = all_nodes.index(hedge_instrument)
            h_hedge[i_start + node_idx] = 1.0
    
    return h_hedge

## 5a. MC Results in pos_summary Format


## 8. Multi-Product Covariance Matrix Analysis

This section uses a shared covariance matrix across all products (cross-product correlations), compared to the independence assumption in sections 6–7.


### 9.3 Execute Hedge Recommendations

### 8a. Build Multi-Product Returns Matrix


In [74]:
# Build multi-product returns matrix
# Collect returns for all products, align dates, filter holidays

print("="*80)
print("BUILDING MULTI-PRODUCT RETURNS MATRIX")
print("="*80)

# Get products that have data
products_with_data = []
product_returns_dict = {}

for mapped_product in all_products:
    product_lower = mapped_to_data_column.get(mapped_product, mapped_product.lower())
    
    # Check if product data exists
    product_cols = [col for col in df_raw.columns if col.startswith(f'{product_lower}_A') and '/' not in col]
    if not product_cols:
        continue
    
    # Extract price data for this product
    product_cols = sorted(product_cols, key=lambda x: int(x.split('_A')[1]))
    df_prices = df_raw[product_cols].copy()
    
    # Compute daily changes (returns)
    df_returns = df_prices.diff().dropna()
    df_returns = df_returns.dropna()
    
    # Filter out holiday dates
    is_holiday = [date.date() in holiday_dates for date in df_returns.index]
    df_returns = df_returns[~pd.Series(is_holiday, index=df_returns.index)]
    
    if len(df_returns) < ewma_init_obs:
        print(f"  WARNING: Insufficient data for {mapped_product} ({len(df_returns)} < {ewma_init_obs}), skipping...")
        continue
    
    products_with_data.append(mapped_product)
    product_returns_dict[mapped_product] = df_returns
    print(f"  {mapped_product}: {len(df_returns)} observations, {len(product_cols)} nodes")

print(f"\nProducts with data: {products_with_data}")
print(f"Total products: {len(products_with_data)}")

# Find common dates across all products
if len(products_with_data) > 0:
    common_dates = product_returns_dict[products_with_data[0]].index
    for product in products_with_data[1:]:
        common_dates = common_dates.intersection(product_returns_dict[product].index)
    
    print(f"\nCommon dates across all products: {len(common_dates)}")
    print(f"Date range: {common_dates.min()} to {common_dates.max()}")
    
    # Build combined returns DataFrame
    combined_returns_list = []
    column_order = []
    
    for product in products_with_data:
        product_lower = mapped_to_data_column.get(product, product.lower())
        product_returns = product_returns_dict[product].loc[common_dates]
        
        # Get columns in order A01-A15
        product_cols = sorted([col for col in product_returns.columns], 
                             key=lambda x: int(x.split('_A')[1]))
        
        for col in product_cols:
            combined_returns_list.append(product_returns[col])
            column_order.append(col)
    
    # Create combined DataFrame
    combined_returns_df = pd.DataFrame(dict(zip(column_order, combined_returns_list)))
    combined_returns_df.index = common_dates
    
    # Create product_indices mapping: product -> (start_idx, end_idx) in combined matrix
    product_indices = {}
    current_idx = 0
    for product in products_with_data:
        product_indices[product] = (current_idx, current_idx + 15)
        current_idx += 15
        print(f"  {product}: columns {product_indices[product][0]}-{product_indices[product][1]-1}")
    
    print(f"\nCombined returns matrix shape: {combined_returns_df.shape}")
    print(f"Columns: {len(column_order)} ({len(products_with_data)} products × 15 nodes)")
    print(f"Rows: {len(common_dates)} (common trading dates)")
    print(f"\nProduct indices mapping created: {len(product_indices)} products")
else:
    print("ERROR: No products with sufficient data!")
    combined_returns_df = pd.DataFrame()
    product_indices = {}


BUILDING MULTI-PRODUCT RETURNS MATRIX
  CLBR: 968 observations, 15 nodes
  HOUBR: 964 observations, 15 nodes
  HTT: 966 observations, 15 nodes
  WDF: 259 observations, 15 nodes

Products with data: ['CLBR', 'HOUBR', 'HTT', 'WDF']
Total products: 4

Common dates across all products: 253
Date range: 2024-12-10 00:00:00 to 2025-12-24 00:00:00
  CLBR: columns 0-14
  HOUBR: columns 15-29
  HTT: columns 30-44
  WDF: columns 45-59

Combined returns matrix shape: (253, 60)
Columns: 60 (4 products × 15 nodes)
Rows: 253 (common trading dates)

Product indices mapping created: 4 products


### 8b. Compute Multi-Product EWMA Covariance Matrix


In [75]:
def compute_multi_product_ewma_covariance(combined_returns_df, products_with_data, product_indices, 
                                           front, mid, back, lambda_front, lambda_mid, lambda_back, 
                                           init_obs=60):
    """
    Compute multi-product EWMA covariance matrix with cross-product correlations.
    Uses bucket-specific lambdas for different tenor buckets.
    
    Parameters:
    -----------
    combined_returns_df : DataFrame
        Combined returns matrix with all products (columns: product_node, e.g., 'htt_A01')
    products_with_data : list
        List of products to include
    product_indices : dict
        Mapping from product to (start_idx, end_idx) in combined matrix
    front, mid, back : list
        Node lists for each bucket
    lambda_front, lambda_mid, lambda_back : float
        EWMA decay parameters for each bucket
    init_obs : int
        Number of observations for initial covariance
    
    Returns:
    --------
    Sigma_multi : ndarray
        Full multi-product covariance matrix
    """
    # Convert to numpy array
    returns_array = combined_returns_df.values  # Shape: (n_dates, n_products * 15)
    n_obs, n_vars = returns_array.shape
    n_products = len(products_with_data)
    
    if n_obs < init_obs:
        raise ValueError(f"Need at least {init_obs} observations, got {n_obs}")
    
    print(f"Computing multi-product EWMA covariance...")
    print(f"  Returns array shape: {returns_array.shape}")
    print(f"  Products: {n_products}")
    print(f"  Total variables: {n_vars} ({n_products} products × 15 nodes)")
    
    # Initialize with sample covariance of first N observations
    init_returns = returns_array[:init_obs]
    # Remove any rows with NaN
    valid_rows = ~np.isnan(init_returns).any(axis=1)
    init_returns = init_returns[valid_rows]
    
    if len(init_returns) < 10:
        # Fallback: use identity matrix scaled by variance
        cov_current = np.eye(n_vars) * np.var(returns_array, axis=0).mean()
        print(f"  WARNING: Using identity matrix initialization (only {len(init_returns)} valid rows)")
    else:
        cov_current = np.cov(init_returns.T)
        print(f"  Initialized with sample covariance from {len(init_returns)} observations")
    
    # Create lambda vector: one lambda per variable based on which bucket its node belongs to
    # Map each variable to its node and determine bucket
    lambda_vec = np.zeros(n_vars)
    
    for product in products_with_data:
        i_start, i_end = product_indices[product]
        product_lower = mapped_to_data_column.get(product, product.lower())
        
        # For each node in this product (A01-A15)
        for node_idx in range(15):
            var_idx = i_start + node_idx
            node_code = all_nodes[node_idx]
            
            # Determine which bucket this node belongs to
            if node_code in front:
                lambda_vec[var_idx] = lambda_front
            elif node_code in mid:
                lambda_vec[var_idx] = lambda_mid
            elif node_code in back:
                lambda_vec[var_idx] = lambda_back
            else:
                lambda_vec[var_idx] = lambda_back  # Default to back
    
    print(f"  Lambda distribution: Front={np.sum(lambda_vec == lambda_front)}, "
          f"Mid={np.sum(lambda_vec == lambda_mid)}, Back={np.sum(lambda_vec == lambda_back)}")
    
    # EWMA recursion: Σ_t = λ * Σ_{t-1} + (1-λ) * r_t * r_t'
    # But we use element-wise lambdas for bucket-specific decay
    for t in range(init_obs, n_obs):
        r_t = returns_array[t:t+1, :]  # Shape: (1, n_vars)
        
        # Skip if any NaN
        if np.isnan(r_t).any():
            continue
        
        # EWMA update with element-wise lambdas
        # For each element (i,j): Σ_{t}[i,j] = λ_ij * Σ_{t-1}[i,j] + (1-λ_ij) * r_t[i] * r_t[j]
        # Use average lambda for off-diagonal: λ_ij = (λ_i + λ_j) / 2
        # For diagonal: use λ_i directly
        
        # Compute outer product
        outer_product = np.outer(r_t[0], r_t[0])  # Shape: (n_vars, n_vars)
        
        # Create lambda matrix: average of row and column lambdas for off-diagonal
        lambda_matrix = np.outer(lambda_vec, np.ones(n_vars))
        lambda_matrix = (lambda_matrix + lambda_matrix.T) / 2  # Average for off-diagonal
        
        # EWMA update
        cov_current = lambda_matrix * cov_current + (1 - lambda_matrix) * outer_product
    
    print(f"  Final covariance matrix shape: {cov_current.shape}")
    print(f"  Completed EWMA recursion for {n_obs - init_obs} observations")
    
    return cov_current

# Compute multi-product EWMA covariance matrix
if len(products_with_data) > 0 and len(combined_returns_df) > 0:
    print("\n" + "="*80)
    print("COMPUTING MULTI-PRODUCT EWMA COVARIANCE MATRIX")
    print("="*80)
    
    Sigma_multi = compute_multi_product_ewma_covariance(
        combined_returns_df, products_with_data, product_indices,
        front, mid, back, lambda_front, lambda_mid, lambda_back, ewma_init_obs
    )
    
    print(f"\nMulti-product covariance matrix computed successfully!")
    print(f"  Shape: {Sigma_multi.shape}")
    print(f"  Size: {Sigma_multi.size:,} elements")
    
    # Check for cross-product correlations (sample)
    if len(products_with_data) >= 2:
        print(f"\nSample cross-product correlations (A01 nodes):")
        for i, prod_i in enumerate(products_with_data):
            for j, prod_j in enumerate(products_with_data):
                if i < j:  # Only show upper triangle
                    i_start, i_end = product_indices[prod_i]
                    j_start, j_end = product_indices[prod_j]
                    
                    # Get A01 node indices (first node)
                    i_node = i_start
                    j_node = j_start
                    
                    # Extract covariance and convert to correlation
                    cov_ij = Sigma_multi[i_node, j_node]
                    var_i = Sigma_multi[i_node, i_node]
                    var_j = Sigma_multi[j_node, j_node]
                    
                    if var_i > 0 and var_j > 0:
                        corr_ij = cov_ij / np.sqrt(var_i * var_j)
                        print(f"  {prod_i} vs {prod_j}: {corr_ij:.4f}")
else:
    print("ERROR: Cannot compute covariance - missing returns data!")
    Sigma_multi = None



COMPUTING MULTI-PRODUCT EWMA COVARIANCE MATRIX
Computing multi-product EWMA covariance...
  Returns array shape: (253, 60)
  Products: 4
  Total variables: 60 (4 products × 15 nodes)
  Initialized with sample covariance from 60 observations
  Lambda distribution: Front=16, Mid=16, Back=28
  Final covariance matrix shape: (60, 60)
  Completed EWMA recursion for 193 observations

Multi-product covariance matrix computed successfully!
  Shape: (60, 60)
  Size: 3,600 elements

Sample cross-product correlations (A01 nodes):
  CLBR vs HOUBR: 0.9100
  CLBR vs HTT: 0.0407
  CLBR vs WDF: -0.6046
  HOUBR vs HTT: 0.4514
  HOUBR vs WDF: -0.7580
  HTT vs WDF: -0.5254


### 8c. Build Combined Position Vectors


In [76]:
# Build combined position vectors for all products
# w_total_combined: combined position vector [product1_A01, ..., product1_A15, product2_A01, ..., product2_A15, ...]

print("="*80)
print("BUILDING COMBINED POSITION VECTORS")
print("="*80)

if len(products_with_data) > 0 and 'product_indices' in globals():
    n_products = len(products_with_data)
    n_combined = n_products * 15  # Each product has 15 nodes
    
    # Initialize combined total position vector
    w_total_combined = np.zeros(n_combined)
    
    print(f"\nBuilding combined position vector for {n_products} products...")
    print(f"  Combined vector size: {n_combined} ({n_products} products × 15 nodes)")
    
    # Build position vector for each product and place in combined vector
    for product in products_with_data:
        if product not in product_indices:
            print(f"  WARNING: {product} not in product_indices, skipping...")
            continue
        
        i_start, i_end = product_indices[product]
        
        # Build single-product position vector
        w_product = np.zeros(15)
        if product in delta_summary_df.columns:
            for _, row in delta_summary_df.iterrows():
                tenor = row['Tenor']
                position = row[product]
                if abs(position) > 1e-10 and tenor in contract_to_node:
                    node = contract_to_node[tenor]
                    node_idx = all_nodes.index(node)
                    w_product[node_idx] = float(position)
        
        # Place in combined vector
        w_total_combined[i_start:i_end] = w_product
        
        total_qty = np.sum(np.abs(w_product))
        print(f"  {product}: columns {i_start}-{i_end-1}, total qty: {total_qty:.2f}")
    
    total_combined_qty = np.sum(np.abs(w_total_combined))
    print(f"\nCombined total position vector built successfully!")
    print(f"  Total absolute quantity across all products: {total_combined_qty:.2f}")
    print(f"  Non-zero elements: {np.sum(np.abs(w_total_combined) > 1e-10)}")
    
    # Display summary by product
    print(f"\nPosition summary by product in combined vector:")
    for product in products_with_data:
        if product in product_indices:
            i_start, i_end = product_indices[product]
            w_product = w_total_combined[i_start:i_end]
            total_qty = np.sum(np.abs(w_product))
            net_qty = np.sum(w_product)
            n_nonzero = np.sum(np.abs(w_product) > 1e-10)
            print(f"  {product}: abs_qty={total_qty:.2f}, net_qty={net_qty:.2f}, non_zero_nodes={n_nonzero}")
else:
    print("ERROR: Cannot build combined position vectors - missing products_with_data or product_indices!")
    w_total_combined = None


BUILDING COMBINED POSITION VECTORS

Building combined position vector for 4 products...
  Combined vector size: 60 (4 products × 15 nodes)
  CLBR: columns 0-14, total qty: 2368.00
  HOUBR: columns 15-29, total qty: 3925.00
  HTT: columns 30-44, total qty: 10549.00
  WDF: columns 45-59, total qty: 145.00

Combined total position vector built successfully!
  Total absolute quantity across all products: 16987.00
  Non-zero elements: 35

Position summary by product in combined vector:
  CLBR: abs_qty=2368.00, net_qty=132.00, non_zero_nodes=5
  HOUBR: abs_qty=3925.00, net_qty=475.00, non_zero_nodes=12
  HTT: abs_qty=10549.00, net_qty=-749.00, non_zero_nodes=15
  WDF: abs_qty=145.00, net_qty=15.00, non_zero_nodes=3


In [77]:
print("="*80)
print("POSITION VECTOR ANALYSIS")
print("="*80)

position_analysis = []

for mapped_product in ['CLBR', 'HOUBR', 'HTT', 'WDF']:
    # Build position vector
    w_total = np.zeros(n_total)
    if mapped_product in delta_summary_df.columns:
        for _, row in delta_summary_df.iterrows():
            tenor = row['Tenor']
            position = row[mapped_product]
            if abs(position) > 1e-10 and tenor in contract_to_node:
                node = contract_to_node[tenor]
                node_idx = all_nodes.index(node)
                w_total[node_idx] = float(position)
    
    total_qty = np.sum(np.abs(w_total))
    net_qty = np.sum(w_total)
    n_nonzero = np.sum(np.abs(w_total) > 1e-10)
    
    # Check which buckets have positions
    front_pos = w_total[[all_nodes.index(n) for n in front]]
    mid_pos = w_total[[all_nodes.index(n) for n in mid]]
    back_pos = w_total[[all_nodes.index(n) for n in back]]
    
    front_qty = np.sum(np.abs(front_pos))
    mid_qty = np.sum(np.abs(mid_pos))
    back_qty = np.sum(np.abs(back_pos))
    
    position_analysis.append({
        'Product': mapped_product,
        'Total_abs_qty': total_qty,
        'Net_qty': net_qty,
        'N_nonzero_nodes': n_nonzero,
        'Front_qty': front_qty,
        'Mid_qty': mid_qty,
        'Back_qty': back_qty,
        'Max_position': np.max(np.abs(w_total)),
        'Max_position_node': all_nodes[np.argmax(np.abs(w_total))],
        'Concentration': np.max(np.abs(w_total)) / total_qty if total_qty > 0 else 0
    })
    
    print(f"\n{mapped_product} Position Analysis:")
    print(f"  Total absolute quantity: {total_qty:.2f}")
    print(f"  Net quantity: {net_qty:.2f}")
    print(f"  Non-zero nodes: {n_nonzero}")
    print(f"  Front bucket qty: {front_qty:.2f}")
    print(f"  Mid bucket qty: {mid_qty:.2f}")
    print(f"  Back bucket qty: {back_qty:.2f}")
    print(f"  Max position: {np.max(np.abs(w_total)):.2f} (node: {all_nodes[np.argmax(np.abs(w_total))]})")
    print(f"  Concentration ratio: {np.max(np.abs(w_total)) / total_qty if total_qty > 0 else 0:.2%}")

pos_analysis_df = pd.DataFrame(position_analysis)
print("\n" + "="*80)
print("POSITION ANALYSIS COMPARISON")
print("="*80)
print(pos_analysis_df.to_string(index=False))


POSITION VECTOR ANALYSIS

CLBR Position Analysis:
  Total absolute quantity: 2368.00
  Net quantity: 132.00
  Non-zero nodes: 5
  Front bucket qty: 268.00
  Mid bucket qty: 2000.00
  Back bucket qty: 100.00
  Max position: 850.00 (node: A07)
  Concentration ratio: 35.90%

HOUBR Position Analysis:
  Total absolute quantity: 3925.00
  Net quantity: 475.00
  Non-zero nodes: 12
  Front bucket qty: 3200.00
  Mid bucket qty: 200.00
  Back bucket qty: 525.00
  Max position: 1300.00 (node: A03)
  Concentration ratio: 33.12%

HTT Position Analysis:
  Total absolute quantity: 10549.00
  Net quantity: -749.00
  Non-zero nodes: 15
  Front bucket qty: 4874.00
  Mid bucket qty: 4600.00
  Back bucket qty: 1075.00
  Max position: 1974.00 (node: A03)
  Concentration ratio: 18.71%

WDF Position Analysis:
  Total absolute quantity: 145.00
  Net quantity: 15.00
  Non-zero nodes: 3
  Front bucket qty: 145.00
  Mid bucket qty: 0.00
  Back bucket qty: 0.00
  Max position: 65.00 (node: A02)
  Concentration ra

### 8d. Recalculate MC with Multi-Product Covariance


In [78]:
# Recalculate MC for each strategy/product using multi-product covariance
if Sigma_multi is not None and w_total_combined is not None:
    print("\n" + "="*80)
    print("RECALCULATING MC WITH MULTI-PRODUCT COVARIANCE")
    print("="*80)
    
    position_mc_multi_product = []
    
    # Process each strategy/product combination
    for result in position_mc_results:
        strategy_name = result['Strategy']
        mapped_product = result['Product']
        
        # Skip if product not in products_with_data
        if mapped_product not in products_with_data:
            continue
        
        # Build combined strategy position vector
        w_strategy_combined = np.zeros(len(w_total_combined))
        
        # Get product index
        if mapped_product in product_indices:
            i_start, i_end = product_indices[mapped_product]
            
            # Build strategy position vector for this product
            w_strategy = build_strategy_position_vector(
                strategy_name, mapped_product, delta_positions_df,
                contract_to_node, all_nodes
            )
            
            # Place in appropriate position in combined vector
            w_strategy_combined[i_start:i_end] = w_strategy
            
            strategy_qty = np.sum(np.abs(w_strategy))
            if strategy_qty < 1e-10:
                continue  # Skip empty strategies
            
            # Calculate MC using multi-product covariance
            # MC = 1000 * (w_strategy' Σ_multi w_total) / sqrt(w_total' Σ_multi w_total)
            numerator = w_strategy_combined.T @ Sigma_multi @ w_total_combined
            denominator = np.sqrt(w_total_combined.T @ Sigma_multi @ w_total_combined)
            
            if denominator > 1e-10:
                mc_multi = 1000.0 * numerator / denominator
            else:
                mc_multi = 0.0
            
            # Get original MC for comparison
            mc_original = result['MC_to_total']
            
            position_mc_multi_product.append({
                'Strategy': strategy_name,
                'Product': mapped_product,
                'MC_original': mc_original,
                'MC_multi_product': mc_multi,
                'Difference': mc_multi - mc_original,
                'Pct_change': ((mc_multi - mc_original) / abs(mc_original) * 100) if abs(mc_original) > 1e-10 else 0.0,
                'Qty_total': strategy_qty
            })
    
    print(f"Recalculated MC for {len(position_mc_multi_product)} strategy/product combinations")
else:
    print("ERROR: Cannot recalculate MC - missing covariance matrix or position vectors!")
    position_mc_multi_product = []



RECALCULATING MC WITH MULTI-PRODUCT COVARIANCE
Recalculated MC for 10 strategy/product combinations


### 8e. MC by Bucket (Shared Covariance)


In [79]:
# Calculate MC by bucket using shared covariance matrix
# For each bucket (Front, Mid, Back), compute MC contribution to total portfolio risk

print("\n" + "="*80)
print("MC BY BUCKET USING SHARED COVARIANCE MATRIX")
print("="*80)

if Sigma_multi is not None and w_total_combined is not None and len(products_with_data) > 0:
    # Build bucket position vectors in combined format
    # Each bucket vector contains positions in that bucket across all products
    
    # Get node indices for each bucket in the combined vector
    n_products = len(products_with_data)
    
    # Build bucket position vectors
    w_front_combined = np.zeros(len(w_total_combined))
    w_mid_combined = np.zeros(len(w_total_combined))
    w_back_combined = np.zeros(len(w_total_combined))
    
    # For each product, extract positions in each bucket
    for product in products_with_data:
        if product not in product_indices:
            continue
        
        i_start, i_end = product_indices[product]
        w_product = w_total_combined[i_start:i_end]
        
        # Map nodes to buckets
        for node_idx in range(15):
            var_idx = i_start + node_idx
            node_code = all_nodes[node_idx]
            
            if node_code in front:
                w_front_combined[var_idx] = w_product[node_idx]
            elif node_code in mid:
                w_mid_combined[var_idx] = w_product[node_idx]
            elif node_code in back:
                w_back_combined[var_idx] = w_product[node_idx]
    
    # Calculate MC for each bucket
    # MC_bucket = 1000 * (w_bucket' Σ_multi w_total) / sqrt(w_total' Σ_multi w_total)
    
    # Compute denominator (total portfolio risk)
    total_var = w_total_combined.T @ Sigma_multi @ w_total_combined
    if total_var <= 0:
        denominator = 1e-10
    else:
        denominator = np.sqrt(total_var)
    
    # Calculate MC for each bucket
    mc_front = 1000.0 * (w_front_combined.T @ Sigma_multi @ w_total_combined) / denominator
    mc_mid = 1000.0 * (w_mid_combined.T @ Sigma_multi @ w_total_combined) / denominator
    mc_back = 1000.0 * (w_back_combined.T @ Sigma_multi @ w_total_combined) / denominator
    
    # Calculate quantities for each bucket
    qty_front = np.sum(np.abs(w_front_combined))
    qty_mid = np.sum(np.abs(w_mid_combined))
    qty_back = np.sum(np.abs(w_back_combined))
    
    # Create summary DataFrame
    mc_by_bucket_shared = pd.DataFrame({
        'Bucket': ['Front', 'Mid', 'Back'],
        'MC_to_total': [mc_front, mc_mid, mc_back],
        'Qty_total': [qty_front, qty_mid, qty_back],
        'MC_signed': ['POS' if x > 0 else 'NEG' if x < 0 else 'ZERO' 
                      for x in [mc_front, mc_mid, mc_back]]
    })
    
    print(f"\nMC by Bucket (using shared covariance matrix):")
    print(mc_by_bucket_shared.to_string(index=False))
    
    print(f"\nTotal portfolio risk (denominator): {denominator:.2f}")
    print(f"Sum of bucket MCs: {mc_front + mc_mid + mc_back:.2f}")
    
    # Also aggregate strategies by bucket
    print(f"\n" + "="*80)
    print("MC BY BUCKET - AGGREGATED FROM STRATEGIES")
    print("="*80)
    
    if len(position_mc_multi_product) > 0:
        # Create DataFrame from multi-product MC results
        mc_multi_df = pd.DataFrame(position_mc_multi_product)
        
        # Add bucket information from original results
        bucket_map = {}
        for result in position_mc_results:
            key = (result['Strategy'], result['Product'])
            bucket_map[key] = result.get('Bucket', 'Unknown')
        
        mc_multi_df['Bucket'] = mc_multi_df.apply(
            lambda row: bucket_map.get((row['Strategy'], row['Product']), 'Unknown'), 
            axis=1
        )
        
        # Aggregate by bucket
        mc_by_bucket_strategies = mc_multi_df.groupby('Bucket').agg({
            'MC_multi_product': 'sum',
            'Qty_total': 'sum'
        }).reset_index()
        mc_by_bucket_strategies.columns = ['Bucket', 'MC_to_total', 'Qty_total']
        mc_by_bucket_strategies['MC_signed'] = mc_by_bucket_strategies['MC_to_total'].apply(
            lambda x: 'POS' if x > 0 else 'NEG' if x < 0 else 'ZERO'
        )
        mc_by_bucket_strategies = mc_by_bucket_strategies.sort_values('MC_to_total', key=abs, ascending=False)
        
        print("\nMC by Bucket (aggregated from strategy/product combinations):")
        print(mc_by_bucket_strategies.to_string(index=False))
        
        # Show breakdown by bucket and product
        print(f"\n" + "="*80)
        print("MC BY BUCKET × PRODUCT (using shared covariance)")
        print("="*80)
        
        # Add product info and aggregate
        mc_multi_df_with_bucket = mc_multi_df.copy()
        mc_bucket_product = mc_multi_df_with_bucket.groupby(['Bucket', 'Product']).agg({
            'MC_multi_product': 'sum',
            'Qty_total': 'sum'
        }).reset_index()
        mc_bucket_product.columns = ['Bucket', 'Product', 'MC_to_total', 'Qty_total']
        mc_bucket_product = mc_bucket_product.sort_values(['Bucket', 'MC_to_total'], ascending=[True, False])
        
        print(mc_bucket_product.to_string(index=False))
    else:
        print("WARNING: No multi-product MC results available for strategy aggregation")
    
else:
    print("ERROR: Cannot calculate MC by bucket - missing covariance matrix, position vectors, or products!")
    mc_by_bucket_shared = pd.DataFrame()



MC BY BUCKET USING SHARED COVARIANCE MATRIX

MC by Bucket (using shared covariance matrix):
Bucket   MC_to_total  Qty_total MC_signed
 Front 130334.457618  8487.0004       POS
   Mid  13469.941541  6800.0000       POS
  Back  10703.128167  1700.0000       POS

Total portfolio risk (denominator): 154.51
Sum of bucket MCs: 154507.53

MC BY BUCKET - AGGREGATED FROM STRATEGIES

MC by Bucket (aggregated from strategy/product combinations):
Bucket   MC_to_total  Qty_total MC_signed
 Mixed 111622.793914 13000.0000       POS
 Front  42884.733412  4050.9996       POS

MC BY BUCKET × PRODUCT (using shared covariance)
Bucket Product  MC_to_total  Qty_total
 Front   HOUBR 28385.233906  3200.0000
 Front     HTT 13268.141929   674.0000
 Front    CLBR   619.695714    31.9996
 Front     WDF   611.661863   145.0000
 Mixed     HTT 96886.453181  9875.0000
 Mixed   HOUBR 12950.160009   725.0000
 Mixed    CLBR  1786.180724  2400.0000


In [80]:
pd.DataFrame(position_mc_multi_product)

,Strategy,Product,MC_original,MC_multi_product,Difference,Pct_change,Qty_total
0,HOUBR_Back,CLBR,-2.186870e+03,619.695714,2806.565324,1.283371e+02,31.9996
1,HOUBR_rolls,CLBR,3.508698e+04,1786.180724,-33300.803923,-9.490928e+01,2400.0000
2,HOUBR_Back,HOUBR,2.155257e+04,12950.160009,-8602.410262,-3.991362e+01,725.0000
3,HOUBR_Front,HOUBR,8.700723e-08,3398.469168,3398.469168,3.905962e+12,600.0000
4,HOUBR_rolls,HOUBR,3.012809e-07,24986.764738,24986.764738,8.293510e+12,2600.0000
5,HTT_Back,HTT,5.510005e+03,1380.447689,-4129.557463,-7.494653e+01,975.0000
6,HTT_Front,HTT,-2.860082e+03,-2046.601924,813.479867,2.844254e+01,100.0000
7,HTT_Mid,HTT,1.641687e+04,15314.743853,-1102.125630,-6.713373e+00,574.0000
8,HTT_Rolls,HTT,1.765134e+05,95506.005492,-81007.360280,-4.589305e+01,8900.0000
9,Freight,WDF,0.000000e+00,611.661863,611.661863,0.000000e+00,145.0000


### 8f. Analysis and Comparison


In [81]:
# Analysis and comparison
if len(position_mc_multi_product) > 0:
    print("\n" + "="*80)
    print("MULTI-PRODUCT COVARIANCE ANALYSIS")
    print("="*80)
    
    # Create comparison DataFrame
    comparison_df = pd.DataFrame(position_mc_multi_product)
    
    # Calculate summary statistics
    total_mc_original = comparison_df['MC_original'].sum()
    total_mc_multi = comparison_df['MC_multi_product'].sum()
    total_difference = total_mc_multi - total_mc_original
    total_pct_change = (total_difference / abs(total_mc_original) * 100) if abs(total_mc_original) > 1e-10 else 0.0
    
    print(f"\nSUMMARY STATISTICS:")
    print(f"  Total MC (independence assumption): {total_mc_original:.2f}")
    print(f"  Total MC (multi-product covariance): {total_mc_multi:.2f}")
    print(f"  Difference: {total_difference:.2f}")
    print(f"  % Change: {total_pct_change:.2f}%")
    
    # Sort by absolute difference
    comparison_df['abs_difference'] = comparison_df['Difference'].abs()
    comparison_df = comparison_df.sort_values('abs_difference', ascending=False)
    
    print(f"\n" + "="*80)
    print("MC COMPARISON BY STRATEGY/PRODUCT")
    print("="*80)
    print(comparison_df[['Strategy', 'Product', 'MC_original', 'MC_multi_product', 
                        'Difference', 'Pct_change']].to_string(index=False))
    
    # Identify most affected strategies
    print(f"\n" + "="*80)
    print("MOST AFFECTED STRATEGIES/PRODUCTS")
    print("="*80)
    significant_changes = comparison_df[comparison_df['abs_difference'] > 100]  # > $100 difference
    if len(significant_changes) > 0:
        print(f"Strategies with difference > $100:")
        print(significant_changes[['Strategy', 'Product', 'MC_original', 'MC_multi_product', 
                                  'Difference', 'Pct_change']].to_string(index=False))
    else:
        print("No strategies with difference > $100")
    
    # Show strategies with >10% change
    large_pct_changes = comparison_df[comparison_df['Pct_change'].abs() > 10]
    if len(large_pct_changes) > 0:
        print(f"\nStrategies with >10% change:")
        print(large_pct_changes[['Strategy', 'Product', 'MC_original', 'MC_multi_product', 
                                'Difference', 'Pct_change']].to_string(index=False))
    else:
        print("\nNo strategies with >10% change")
    
    # Covariance matrix insights
    print(f"\n" + "="*80)
    print("CROSS-PRODUCT CORRELATION INSIGHTS")
    print("="*80)
    
    if len(products_with_data) >= 2:
        print("\nSample cross-product correlations (A01 nodes):")
        for i, prod_i in enumerate(products_with_data):
            for j, prod_j in enumerate(products_with_data):
                if i < j:  # Only show upper triangle
                    i_start, i_end = product_indices[prod_i]
                    j_start, j_end = product_indices[prod_j]
                    
                    # Get A01 node indices (first node)
                    i_node = i_start
                    j_node = j_start
                    
                    # Extract covariance and convert to correlation
                    cov_ij = Sigma_multi[i_node, j_node]
                    var_i = Sigma_multi[i_node, i_node]
                    var_j = Sigma_multi[j_node, j_node]
                    
                    if var_i > 0 and var_j > 0:
                        corr_ij = cov_ij / np.sqrt(var_i * var_j)
                        print(f"  {prod_i} vs {prod_j}: {corr_ij:.4f}")
        
        # Show average cross-product correlation
        cross_corrs = []
        for i, prod_i in enumerate(products_with_data):
            for j, prod_j in enumerate(products_with_data):
                if i != j:
                    i_start, i_end = product_indices[prod_i]
                    j_start, j_end = product_indices[prod_j]
                    
                    # Get diagonal of cross-product block
                    cross_block = Sigma_multi[i_start:i_end, j_start:j_end]
                    var_i_block = np.diag(Sigma_multi[i_start:i_end, i_start:i_end])
                    var_j_block = np.diag(Sigma_multi[j_start:j_end, j_start:j_end])
                    
                    # Average correlation across nodes
                    corrs = []
                    for k in range(15):
                        if var_i_block[k] > 0 and var_j_block[k] > 0:
                            corr_k = cross_block[k, k] / np.sqrt(var_i_block[k] * var_j_block[k])
                            corrs.append(corr_k)
                    
                    if len(corrs) > 0:
                        cross_corrs.append(np.mean(corrs))
        
        if len(cross_corrs) > 0:
            print(f"\nAverage cross-product correlation: {np.mean(cross_corrs):.4f}")
            print(f"  Min: {np.min(cross_corrs):.4f}")
            print(f"  Max: {np.max(cross_corrs):.4f}")
    


    # Bucket-level comparison (independent vs shared, direct vs aggregated)
    print(f"\n" + "="*80)
    print("MC BY BUCKET COMPARISON")
    print("="*80)
    
    bucket_map = {}
    for result in position_mc_results:
        key = (result['Strategy'], result['Product'])
        bucket_map[key] = result.get('Bucket', 'Unknown')
    
    comparison_df['Bucket'] = comparison_df.apply(
        lambda row: bucket_map.get((row['Strategy'], row['Product']), 'Unknown'), 
        axis=1
    )
    
    mc_by_bucket_original = comparison_df.groupby('Bucket').agg({'MC_original': 'sum'}).reset_index()
    mc_by_bucket_original.columns = ['Bucket', 'MC_independent']
    mc_by_bucket_multi = comparison_df.groupby('Bucket').agg({'MC_multi_product': 'sum'}).reset_index()
    mc_by_bucket_multi.columns = ['Bucket', 'MC_shared']
    mc_bucket_comparison = mc_by_bucket_original.merge(mc_by_bucket_multi, on='Bucket', how='outer').fillna(0.0)
    mc_bucket_comparison['Difference'] = mc_bucket_comparison['MC_shared'] - mc_bucket_comparison['MC_independent']
    mc_bucket_comparison['Pct_change'] = (mc_bucket_comparison['Difference'] / mc_bucket_comparison['MC_independent'].abs() * 100).replace([np.inf, -np.inf], 0.0)
    mc_bucket_comparison = mc_bucket_comparison.sort_values('Bucket')
    
    print("\nMC by Bucket - Independent vs Shared Covariance:")
    print(mc_bucket_comparison.to_string(index=False))
    
    print("\nBucket-level impact of cross-product correlations:")
    for _, row in mc_bucket_comparison.iterrows():
        if abs(row['Difference']) > 1:
            print(f"  {row['Bucket']}: {row['MC_independent']:.2f} -> {row['MC_shared']:.2f} ({row['Difference']:+.2f}, {row['Pct_change']:+.2f}%)")
    
    if 'mc_by_bucket_shared' in globals() and len(mc_by_bucket_shared) > 0:
        print("\n" + "="*80)
        print("DIRECT BUCKET MC vs AGGREGATED (from strategies)")
        print("="*80)
        bucket_direct_vs_agg = mc_by_bucket_shared[['Bucket', 'MC_to_total']].copy()
        bucket_direct_vs_agg.columns = ['Bucket', 'MC_direct']
        bucket_direct_vs_agg = bucket_direct_vs_agg.merge(mc_bucket_comparison[['Bucket', 'MC_shared']], on='Bucket', how='outer').fillna(0.0)
        bucket_direct_vs_agg['Difference'] = bucket_direct_vs_agg['MC_direct'] - bucket_direct_vs_agg['MC_shared']
        print(bucket_direct_vs_agg.to_string(index=False))
        print("\nNote: Direct = bucket position vectors; Aggregated = sum of strategy/product MCs by bucket.")

    print(f"\n" + "="*80)
    print("INTERPRETATION")
    print("="*80)
    print("""
The multi-product covariance matrix accounts for cross-product correlations:
- Positive correlations: Products move together, increasing total portfolio risk
- Negative correlations: Products move in opposite directions, reducing total risk (diversification benefit)
- The difference between independence and multi-product MC shows the impact of these correlations
- Bucket-level analysis shows how cross-product correlations affect risk in different tenor buckets
    """)
else:
    print("ERROR: No multi-product MC results to analyze!")



MULTI-PRODUCT COVARIANCE ANALYSIS

SUMMARY STATISTICS:
  Total MC (independence assumption): 250032.84
  Total MC (multi-product covariance): 154507.53
  Difference: -95525.32
  % Change: -38.21%

MC COMPARISON BY STRATEGY/PRODUCT
   Strategy Product   MC_original  MC_multi_product    Difference    Pct_change
  HTT_Rolls     HTT  1.765134e+05      95506.005492 -81007.360280 -4.589305e+01
HOUBR_rolls    CLBR  3.508698e+04       1786.180724 -33300.803923 -9.490928e+01
HOUBR_rolls   HOUBR  3.012809e-07      24986.764738  24986.764738  8.293510e+12
 HOUBR_Back   HOUBR  2.155257e+04      12950.160009  -8602.410262 -3.991362e+01
   HTT_Back     HTT  5.510005e+03       1380.447689  -4129.557463 -7.494653e+01
HOUBR_Front   HOUBR  8.700723e-08       3398.469168   3398.469168  3.905962e+12
 HOUBR_Back    CLBR -2.186870e+03        619.695714   2806.565324  1.283371e+02
    HTT_Mid     HTT  1.641687e+04      15314.743853  -1102.125630 -6.713373e+00
  HTT_Front     HTT -2.860082e+03      -2046.601

## 9. Final Summary


In [82]:
print("="*80)
print("POSITION MC REPORT - FINAL SUMMARY")
print("="*80)

print("\n1. BY STRATEGY:")
print(mc_by_strategy.to_string(index=False))

print("\n\n2. BY PRODUCT:")
print(mc_by_product.to_string(index=False))

print("\n\n3. DETAILED POSITION MC (first 20 rows):")
print(position_mc_df.head(20).to_string(index=False))

print("\n\n4. STRATEGY × PRODUCT PIVOT:")
print(strategy_product_pivot.to_string())

print("\n\n" + "="*80)
print("Report generation complete.")
print("="*80)


POSITION MC REPORT - FINAL SUMMARY

1. BY STRATEGY:
   Strategy   MC_to_total  Qty_total  Tenor_count    Products MC_signed
  HTT_Rolls  1.765134e+05  8900.0000           30         HTT       POS
HOUBR_rolls  3.508698e+04  5000.0000            8 CLBR, HOUBR       POS
 HOUBR_Back  1.936570e+04   756.9996           16 CLBR, HOUBR       POS
    HTT_Mid  1.641687e+04   574.0000            1         HTT       POS
   HTT_Back  5.510005e+03   975.0000           18         HTT       POS
  HTT_Front -2.860082e+03   100.0000            1         HTT       NEG
HOUBR_Front  8.700723e-08   600.0000            2       HOUBR       POS
    Freight  0.000000e+00   145.0000            3         WDF      ZERO


2. BY PRODUCT:
Product   MC_to_total  Qty_total  Tenor_count  Strategy_count MC_signed
    HTT 195580.158615 10549.0000           50               4       POS
   CLBR  32900.115037  2431.9996            7               2       POS
  HOUBR  21552.570271  3925.0000           19               3      

## 9. Portfolio-Wide Hedge Recommender

This section recommends hedge instruments from HTT or CLBR to reduce total portfolio risk using the shared covariance matrix. Unlike the single-product hedge recommender in `q_risk_report.ipynb`, this evaluates hedges at the portfolio level, accounting for cross-product correlations.

### 9.1 Helper Functions

In [83]:
def build_hedge_universe_for_product(product, front_nodes, mid_nodes, back_nodes):
    """
    Build hedge universe for a given product (all outrights + spreads).
    
    Parameters:
    -----------
    product : str
        Product name (e.g., 'HTT', 'CLBR')
    front_nodes, mid_nodes, back_nodes : list
        Node lists for each bucket
    
    Returns:
    --------
    hedge_instruments : list
        List of instrument names (outrights and spreads)
    """
    hedge_instruments = []
    
    # Add all 15 outright nodes (A01-A15)
    all_nodes_list = [f"A{i:02d}" for i in range(1, 16)]
    for node in all_nodes_list:
        hedge_instruments.append(node)
    
    # Add adjacent spreads within front bucket
    for i in range(len(front_nodes) - 1):
        spread_name = f"{front_nodes[i]}/{front_nodes[i+1]}"
        hedge_instruments.append(spread_name)
    
    # Add adjacent spreads within mid bucket
    for i in range(len(mid_nodes) - 1):
        spread_name = f"{mid_nodes[i]}/{mid_nodes[i+1]}"
        hedge_instruments.append(spread_name)
    
    # Add adjacent spreads within back bucket
    for i in range(len(back_nodes) - 1):
        spread_name = f"{back_nodes[i]}/{back_nodes[i+1]}"
        hedge_instruments.append(spread_name)
    
    # For back bucket, add longer/coarse spreads
    if len(back_nodes) >= 4:
        hedge_instruments.append(f"{back_nodes[0]}/{back_nodes[3]}")  # A09/A12
    if len(back_nodes) >= 6:
        hedge_instruments.append(f"{back_nodes[2]}/{back_nodes[5]}")  # A11/A14
    
    return hedge_instruments

In [84]:
def build_hedge_vector(hedge_instrument, hedge_product, product_indices, all_nodes, n_combined):
    """
    Convert a hedge instrument (node or spread) into a position vector in the combined space.
    
    Parameters:
    -----------
    hedge_instrument : str
        Instrument name, either outright (e.g., 'A01') or spread (e.g., 'A01/A02')
    hedge_product : str
        Product name ('HTT' or 'CLBR')
    product_indices : dict
        Dictionary mapping product -> (start_idx, end_idx) in combined space
    all_nodes : list
        List of all node codes ['A01', ..., 'A15']
    n_combined : int
        Total length of combined vector
    
    Returns:
    --------
    h_hedge : ndarray
        Position vector of length n_combined with hedge position
    """
    # Initialize zero vector
    h_hedge = np.zeros(n_combined)
    
    # Check if hedge product exists in product_indices
    if hedge_product not in product_indices:
        return h_hedge
    
    # Get product's slice in combined space
    i_start, i_end = product_indices[hedge_product]
    
    # Check if it's a spread (contains '/')
    if '/' in hedge_instrument:
        # Spread: e.g., 'A01/A02' means long A01, short A02
        parts = hedge_instrument.split('/')
        if len(parts) != 2:
            return h_hedge  # Invalid spread format
        
        node1, node2 = parts[0], parts[1]
        
        # Find node indices
        if node1 in all_nodes and node2 in all_nodes:
            idx1 = all_nodes.index(node1)
            idx2 = all_nodes.index(node2)
            
            # Long first node, short second node
            h_hedge[i_start + idx1] = 1.0
            h_hedge[i_start + idx2] = -1.0
    else:
        # Outright: e.g., 'A01'
        if hedge_instrument in all_nodes:
            node_idx = all_nodes.index(hedge_instrument)
            h_hedge[i_start + node_idx] = 1.0
    
    return h_hedge

In [85]:
def recommend_portfolio_hedge(Sigma_multi, w_total_combined, product_indices, hedge_products, 
                               all_nodes, front, mid, back):
    """
    Recommend hedge instruments that reduce total portfolio risk.
    
    Parameters:
    -----------
    Sigma_multi : ndarray
        Multi-product covariance matrix (n_combined × n_combined)
    w_total_combined : ndarray
        Current portfolio position vector (length n_combined)
    product_indices : dict
        Dictionary mapping product -> (start_idx, end_idx) in combined space
    hedge_products : list
        List of products to use as hedges (e.g., ['HTT', 'CLBR'])
    all_nodes : list
        List of all node codes ['A01', ..., 'A15']
    front, mid, back : list
        Bucket node definitions
    
    Returns:
    --------
    hedge_recommendations_df : DataFrame
        DataFrame sorted by risk_reduction (descending)
        Columns: hedge_instrument, product, beta, risk_reduction, 
                 current_risk, hedged_risk, recommended_lots
    """
    # Compute current portfolio risk
    current_risk = np.sqrt(w_total_combined.T @ Sigma_multi @ w_total_combined)
    
    n_combined = len(w_total_combined)
    hedge_results = []
    
    # For each hedge product (HTT, CLBR)
    for hedge_product in hedge_products:
        if hedge_product not in product_indices:
            print(f"  WARNING: {hedge_product} not in product_indices, skipping...")
            continue
        
        # Build hedge universe for this product
        hedge_universe = build_hedge_universe_for_product(hedge_product, front, mid, back)
        
        print(f"  Evaluating {len(hedge_universe)} hedge instruments from {hedge_product}...")
        
        # For each hedge instrument
        for hedge_instrument in hedge_universe:
            # Build hedge vector in combined space
            h_hedge = build_hedge_vector(hedge_instrument, hedge_product, product_indices, 
                                        all_nodes, n_combined)
            
            # Skip if hedge vector is all zeros (invalid instrument)
            if np.sum(np.abs(h_hedge)) < 1e-10:
                continue
            
            # Compute h'Σh (variance of hedge instrument)
            h_Σ_h = h_hedge.T @ Sigma_multi @ h_hedge
            
            # Skip if invalid variance
            if h_Σ_h <= 0:
                continue
            
            # Compute w'Σh (covariance between portfolio and hedge)
            w_Σ_h = w_total_combined.T @ Sigma_multi @ h_hedge
            
            # Compute optimal beta: β = -(w'Σh) / (h'Σh)
            # This minimizes portfolio variance: Var(w + βh) = w'Σw + 2β(w'Σh) + β²(h'Σh)
            beta = -w_Σ_h / h_Σ_h
            
            # Compute hedged portfolio
            w_hedged = w_total_combined + beta * h_hedge
            
            # Compute hedged portfolio risk
            hedged_risk = np.sqrt(w_hedged.T @ Sigma_multi @ w_hedged)
            
            # Compute risk reduction
            risk_reduction = (current_risk - hedged_risk) / current_risk if current_risk > 0 else 0.0
            
            # Compute recommended lots
            # Beta is the optimal hedge ratio: units of hedge per unit of hedge vector
            # For outrights (hedge_magnitude = 1.0): beta is already in lots
            # For spreads (hedge_magnitude = 2.0): beta gives the spread quantity
            hedge_magnitude = np.sum(np.abs(h_hedge))
            
            if hedge_magnitude > 1e-10:
                # Beta is already in the correct units for the hedge instrument
                # For outrights: beta directly gives recommended lots
                # For spreads: beta gives the spread quantity (already correct)
                recommended_lots = beta
            else:
                recommended_lots = 0.0
            
            hedge_results.append({
                'hedge_instrument': hedge_instrument,
                'product': hedge_product,
                'beta': beta,
                'risk_reduction': risk_reduction,
                'current_risk': current_risk,
                'hedged_risk': hedged_risk,
                'recommended_lots': recommended_lots
            })
    
    # Create DataFrame and sort by risk reduction
    hedge_df = pd.DataFrame(hedge_results)
    if len(hedge_df) > 0:
        hedge_df = hedge_df.sort_values('risk_reduction', ascending=False)
    
    return hedge_df

### 9.2 Main Recommender Function

In [86]:
# Execute portfolio-wide hedge recommendations
print("="*80)
print("PORTFOLIO-WIDE HEDGE RECOMMENDATIONS")
print("="*80)

if Sigma_multi is not None and w_total_combined is not None and 'product_indices' in globals():
    # Get current portfolio risk
    current_risk = np.sqrt(w_total_combined.T @ Sigma_multi @ w_total_combined)
    print(f"\nCurrent Portfolio Risk: {current_risk:.2f}")
    print(f"\nEvaluating hedge instruments from HTT and CLBR...")
    
    # Call recommender function
    hedge_recommendations = recommend_portfolio_hedge(
        Sigma_multi=Sigma_multi,
        w_total_combined=w_total_combined,
        product_indices=product_indices,
        hedge_products=['HTT', 'CLBR'],
        all_nodes=all_nodes,
        front=front,
        mid=mid,
        back=back
    )
    
    if len(hedge_recommendations) > 0:
        print(f"\nEvaluated {len(hedge_recommendations)} hedge instruments")
        
        # Display top 20 recommendations
        top_n = min(20, len(hedge_recommendations))
        top_recommendations = hedge_recommendations.head(top_n)
        
        print(f"\n{'='*80}")
        print(f"TOP {top_n} HEDGE RECOMMENDATIONS")
        print(f"{'='*80}")
        
        # Format output columns
        display_cols = ['hedge_instrument', 'product', 'beta', 'risk_reduction', 
                        'current_risk', 'hedged_risk', 'recommended_lots']
        display_df = top_recommendations[display_cols].copy()
        
        # Format numeric columns for display
        display_df['beta'] = display_df['beta'].apply(lambda x: f"{x:.4f}")
        display_df['risk_reduction'] = display_df['risk_reduction'].apply(lambda x: f"{x:.4%}")
        display_df['current_risk'] = display_df['current_risk'].apply(lambda x: f"{x:.2f}")
        display_df['hedged_risk'] = display_df['hedged_risk'].apply(lambda x: f"{x:.2f}")
        display_df['recommended_lots'] = display_df['recommended_lots'].apply(lambda x: f"{x:.2f}")
        
        print(display_df.to_string(index=False))
        
        # Summary statistics
        print(f"\n{'='*80}")
        print("SUMMARY STATISTICS")
        print(f"{'='*80}")
        print(f"Total hedge instruments evaluated: {len(hedge_recommendations)}")
        print(f"Best risk reduction: {hedge_recommendations['risk_reduction'].max():.4%}")
        print(f"Average risk reduction (top 10): {hedge_recommendations.head(10)['risk_reduction'].mean():.4%}")
        print(f"Number of hedges with >1% risk reduction: {len(hedge_recommendations[hedge_recommendations['risk_reduction'] > 0.01])}")
        print(f"Number of hedges with >5% risk reduction: {len(hedge_recommendations[hedge_recommendations['risk_reduction'] > 0.05])}")
        
        # Breakdown by product
        print(f"\n{'='*80}")
        print("BREAKDOWN BY HEDGE PRODUCT")
        print(f"{'='*80}")
        for product in ['HTT', 'CLBR']:
            product_hedges = hedge_recommendations[hedge_recommendations['product'] == product]
            if len(product_hedges) > 0:
                print(f"\n{product}:")
                print(f"  Total instruments: {len(product_hedges)}")
                print(f"  Best risk reduction: {product_hedges['risk_reduction'].max():.4%}")
                print(f"  Top instrument: {product_hedges.iloc[0]['hedge_instrument']} "
                      f"(risk reduction: {product_hedges.iloc[0]['risk_reduction']:.4%}, "
                      f"beta: {product_hedges.iloc[0]['beta']:.4f})")
    else:
        print("ERROR: No hedge recommendations generated!")
else:
    print("ERROR: Missing required variables (Sigma_multi, w_total_combined, or product_indices)!")

PORTFOLIO-WIDE HEDGE RECOMMENDATIONS

Current Portfolio Risk: 154.51

Evaluating hedge instruments from HTT and CLBR...
  Evaluating 29 hedge instruments from HTT...
  Evaluating 29 hedge instruments from CLBR...

Evaluated 58 hedge instruments

TOP 20 HEDGE RECOMMENDATIONS
hedge_instrument product       beta risk_reduction current_risk hedged_risk recommended_lots
             A03     HTT  3135.5505       32.2840%       154.51      104.63          3135.55
             A02     HTT  2244.7161       25.3719%       154.51      115.31          2244.72
         A03/A04     HTT  3441.7143       21.5224%       154.51      121.25          3441.71
             A01     HTT  1178.7809        8.1382%       154.51      141.93          1178.78
             A08    CLBR  -935.2062        7.5998%       154.51      142.77          -935.21
             A07    CLBR  -897.7153        7.0331%       154.51      143.64          -897.72
             A09    CLBR  -916.4144        6.8735%       154.51      143.8

### 9.3 Execute Hedge Recommendations

## 10. Publishing Tables

This section consolidates and exports the key analysis tables for reporting and downstream consumption.

In [87]:
# -----------------------------------------------------------------------------
# Table 7: MC by Strategy (Multi-Product Covariance) - Aggregated across products
# -----------------------------------------------------------------------------
print("\n" + "="*80)
print("TABLE 7: MC BY STRATEGY (Multi-Product Covariance)")
print("="*80)

if len(position_mc_multi_product) > 0:
    # Create DataFrame and aggregate by Strategy
    mc_multi_df = pd.DataFrame(position_mc_multi_product)
    
    mc_by_strategy_multi = mc_multi_df.groupby('Strategy').agg({
        'MC_original': 'sum',
        'MC_multi_product': 'sum',
        'Qty_total': 'sum'
    }).reset_index()
    
    # Calculate difference and percentage change
    mc_by_strategy_multi['Difference'] = mc_by_strategy_multi['MC_multi_product'] - mc_by_strategy_multi['MC_original']
    mc_by_strategy_multi['Pct_change'] = (mc_by_strategy_multi['Difference'] / mc_by_strategy_multi['MC_original'].abs() * 100).replace([np.inf, -np.inf], 0.0)
    
    # Sort by absolute MC (multi-product)
    mc_by_strategy_multi = mc_by_strategy_multi.sort_values('MC_multi_product', key=abs, ascending=False).reset_index(drop=True)
    
    # Display raw values
    print("\nRaw values:")
    print(mc_by_strategy_multi.to_string(index=False))
    
    # Display formatted version
    print("\nFormatted:")
    table_strategy_multi = mc_by_strategy_multi.copy()
    table_strategy_multi['MC_original'] = table_strategy_multi['MC_original'].apply(lambda x: f"${x:,.2f}")
    table_strategy_multi['MC_multi_product'] = table_strategy_multi['MC_multi_product'].apply(lambda x: f"${x:,.2f}")
    table_strategy_multi['Difference'] = table_strategy_multi['Difference'].apply(lambda x: f"${x:+,.2f}")
    table_strategy_multi['Pct_change'] = table_strategy_multi['Pct_change'].apply(lambda x: f"{x:+.2f}%")
    table_strategy_multi['Qty_total'] = table_strategy_multi['Qty_total'].apply(lambda x: f"{x:,.2f}")
    print(table_strategy_multi.to_string(index=False))
    
    # Summary
    print(f"\nTotal MC (Independence): ${mc_by_strategy_multi['MC_original'].apply(lambda x: float(x.replace('$','').replace(',',''))).sum():,.2f}" if isinstance(mc_by_strategy_multi['MC_original'].iloc[0], str) else f"\nTotal MC (Independence): ${mc_multi_df['MC_original'].sum():,.2f}")
    print(f"Total MC (Multi-Product): ${mc_multi_df['MC_multi_product'].sum():,.2f}")
else:
    print("(Not available - run multi-product covariance analysis first)")


TABLE 7: MC BY STRATEGY (Multi-Product Covariance)

Raw values:
   Strategy   MC_original  MC_multi_product  Qty_total    Difference    Pct_change
  HTT_Rolls  1.765134e+05      95506.005492  8900.0000 -81007.360280 -4.589305e+01
HOUBR_rolls  3.508698e+04      26772.945462  5000.0000  -8314.039186 -2.369551e+01
    HTT_Mid  1.641687e+04      15314.743853   574.0000  -1102.125630 -6.713373e+00
 HOUBR_Back  1.936570e+04      13569.855723   756.9996  -5795.844938 -2.992840e+01
HOUBR_Front  8.700723e-08       3398.469168   600.0000   3398.469168  3.905962e+12
  HTT_Front -2.860082e+03      -2046.601924   100.0000    813.479867  2.844254e+01
   HTT_Back  5.510005e+03       1380.447689   975.0000  -4129.557463 -7.494653e+01
    Freight  0.000000e+00        611.661863   145.0000    611.661863  0.000000e+00

Formatted:
   Strategy MC_original MC_multi_product Qty_total  Difference         Pct_change
  HTT_Rolls $176,513.37       $95,506.01  8,900.00 $-81,007.36            -45.89%
HOUBR_rolls 

In [88]:
# =============================================================================
# FUNCTION: compute_strategy_mc_table
# =============================================================================
# This function produces the MC by Strategy table using multi-product covariance
# It only depends on the core inputs of this notebook
# =============================================================================

def compute_strategy_mc_table(
    df_raw,
    delta_positions_df,
    delta_summary_df,
    holiday_dates,
    # Configuration parameters with defaults
    front=["A01", "A02", "A03", "A04"],
    mid=["A05", "A06", "A07", "A08"],
    back=["A09", "A10", "A11", "A12", "A13", "A14", "A15"],
    lambda_front=0.97,
    lambda_mid=0.98,
    lambda_back=0.99,
    ewma_init_obs=60,
    mapped_to_data_column=None
):
    """
    Compute MC by Strategy table using multi-product covariance.
    
    Parameters:
    -----------
    df_raw : DataFrame
        Raw price data with date index and columns like 'htt_A01', 'houbr_A02', etc.
    delta_positions_df : DataFrame
        Expanded positions with columns: Strategy, Mapped_Product, Tenor, Qty
    delta_summary_df : DataFrame
        Summary of deltas by tenor with columns: Tenor, and product columns (HTT, HOUBR, etc.)
    holiday_dates : set
        Set of holiday dates to exclude from returns calculation
    front, mid, back : list
        Node definitions for each bucket
    lambda_front, lambda_mid, lambda_back : float
        EWMA decay parameters for each bucket
    ewma_init_obs : int
        Number of observations for EWMA initialization
    mapped_to_data_column : dict
        Mapping from Mapped_Product to data column prefix (e.g., {'HTT': 'htt'})
    
    Returns:
    --------
    mc_by_strategy_multi : DataFrame
        DataFrame with columns: Strategy, MC_original, MC_multi_product, Difference, Pct_change, Qty_total
    """
    
    # Default mapping if not provided
    if mapped_to_data_column is None:
        mapped_to_data_column = {
            'HTT': 'htt',
            'HOUBR': 'houbr',
            'CLBR': 'clbr',
            'WDF': 'wdf',
            'LH': 'lh'
        }
    
    # All nodes
    all_nodes = [f"A{i:02d}" for i in range(1, 16)]
    n_total = len(all_nodes)
    
    # Build contract-to-node mapping
    unique_tenors = delta_summary_df['Tenor'].unique()
    contract_to_node = {}
    for idx, tenor in enumerate(unique_tenors[:15]):
        node_code = f"A{idx+1:02d}"
        contract_to_node[tenor] = node_code
    
    # Get all unique mapped products
    all_products = sorted(delta_positions_df['Mapped_Product'].unique())
    
    # -------------------------------------------------------------------------
    # STEP 1: Build multi-product returns matrix
    # -------------------------------------------------------------------------
    products_with_data = []
    product_returns_dict = {}
    
    for mapped_product in all_products:
        product_lower = mapped_to_data_column.get(mapped_product, mapped_product.lower())
        
        # Check if product data exists
        product_cols = [col for col in df_raw.columns if col.startswith(f'{product_lower}_A') and '/' not in col]
        if not product_cols:
            continue
        
        # Extract price data
        product_cols = sorted(product_cols, key=lambda x: int(x.split('_A')[1]))
        df_prices = df_raw[product_cols].copy()
        
        # Compute daily changes
        df_returns = df_prices.diff().dropna()
        
        # Filter out holiday dates
        is_holiday = [date.date() in holiday_dates for date in df_returns.index]
        df_returns = df_returns[~pd.Series(is_holiday, index=df_returns.index)]
        
        if len(df_returns) < ewma_init_obs:
            continue
        
        products_with_data.append(mapped_product)
        product_returns_dict[mapped_product] = df_returns
    
    if len(products_with_data) == 0:
        raise ValueError("No products with sufficient data found")
    
    # Find common dates
    common_dates = product_returns_dict[products_with_data[0]].index
    for product in products_with_data[1:]:
        common_dates = common_dates.intersection(product_returns_dict[product].index)
    
    if len(common_dates) < ewma_init_obs:
        raise ValueError(f"Insufficient common dates: {len(common_dates)} < {ewma_init_obs}")
    
    # Build combined returns DataFrame
    combined_returns_list = []
    column_order = []
    
    for product in products_with_data:
        product_lower = mapped_to_data_column.get(product, product.lower())
        product_returns = product_returns_dict[product].loc[common_dates]
        product_cols = sorted([col for col in product_returns.columns], 
                             key=lambda x: int(x.split('_A')[1]))
        
        for col in product_cols:
            combined_returns_list.append(product_returns[col])
            column_order.append(col)
    
    combined_returns_df = pd.DataFrame(dict(zip(column_order, combined_returns_list)))
    combined_returns_df.index = common_dates
    
    # Product indices mapping
    product_indices = {}
    current_idx = 0
    for product in products_with_data:
        product_indices[product] = (current_idx, current_idx + 15)
        current_idx += 15
    
    n_combined = len(products_with_data) * 15
    
    # -------------------------------------------------------------------------
    # STEP 2: Compute multi-product EWMA covariance matrix
    # -------------------------------------------------------------------------
    returns_array = combined_returns_df.values
    n_obs, n_vars = returns_array.shape
    
    # Initialize with sample covariance
    init_returns = returns_array[:ewma_init_obs]
    valid_rows = ~np.isnan(init_returns).any(axis=1)
    init_returns = init_returns[valid_rows]
    
    if len(init_returns) < 10:
        cov_current = np.eye(n_vars) * np.var(returns_array, axis=0).mean()
    else:
        cov_current = np.cov(init_returns.T)
    
    # Create lambda vector
    lambda_vec = np.zeros(n_vars)
    for product in products_with_data:
        i_start, i_end = product_indices[product]
        for node_idx in range(15):
            var_idx = i_start + node_idx
            node_code = all_nodes[node_idx]
            if node_code in front:
                lambda_vec[var_idx] = lambda_front
            elif node_code in mid:
                lambda_vec[var_idx] = lambda_mid
            else:
                lambda_vec[var_idx] = lambda_back
    
    # EWMA recursion
    for t in range(ewma_init_obs, n_obs):
        r_t = returns_array[t:t+1, :]
        if np.isnan(r_t).any():
            continue
        outer_product = np.outer(r_t[0], r_t[0])
        lambda_matrix = np.outer(lambda_vec, np.ones(n_vars))
        lambda_matrix = (lambda_matrix + lambda_matrix.T) / 2
        cov_current = lambda_matrix * cov_current + (1 - lambda_matrix) * outer_product
    
    Sigma_multi = cov_current
    
    # -------------------------------------------------------------------------
    # STEP 3: Build combined position vector
    # -------------------------------------------------------------------------
    w_total_combined = np.zeros(n_combined)
    
    for product in products_with_data:
        if product not in product_indices:
            continue
        i_start, i_end = product_indices[product]
        
        w_product = np.zeros(15)
        if product in delta_summary_df.columns:
            for _, row in delta_summary_df.iterrows():
                tenor = row['Tenor']
                position = row[product]
                if abs(position) > 1e-10 and tenor in contract_to_node:
                    node = contract_to_node[tenor]
                    node_idx = all_nodes.index(node)
                    w_product[node_idx] = float(position)
        
        w_total_combined[i_start:i_end] = w_product
    
    # -------------------------------------------------------------------------
    # STEP 4: Calculate single-product MC (independence assumption)
    # -------------------------------------------------------------------------
    def compute_single_product_ewma_cov(returns_df, nodes, product_lower, lambda_val, init_obs):
        cols = [f'{product_lower}_{node}' for node in nodes]
        returns_subset = returns_df[cols].values
        n_obs_sp, n_nodes = returns_subset.shape
        
        init_ret = returns_subset[:init_obs]
        init_ret = init_ret[~np.isnan(init_ret).any(axis=1)]
        
        if len(init_ret) < 10:
            cov_sp = np.eye(n_nodes) * np.var(returns_subset, axis=0).mean()
        else:
            cov_sp = np.cov(init_ret.T)
        
        for t in range(init_obs, n_obs_sp):
            r_t = returns_subset[t:t+1, :]
            if np.isnan(r_t).any():
                continue
            outer_prod = np.outer(r_t[0], r_t[0])
            cov_sp = lambda_val * cov_sp + (1 - lambda_val) * outer_prod
        
        return cov_sp
    
    def build_strategy_position_vector(strategy_name, product, delta_pos_df):
        strategy_positions = delta_pos_df[
            (delta_pos_df['Strategy'] == strategy_name) & 
            (delta_pos_df['Mapped_Product'] == product)
        ].copy()
        
        w_strategy = np.zeros(15)
        for _, row in strategy_positions.iterrows():
            tenor = row['Tenor']
            qty = row['Qty']
            if tenor in contract_to_node:
                node = contract_to_node[tenor]
                node_idx = all_nodes.index(node)
                w_strategy[node_idx] += qty
        
        return w_strategy
    
    # Calculate MC for each strategy/product combination
    position_mc_results = []
    
    for mapped_product in products_with_data:
        product_lower = mapped_to_data_column.get(mapped_product, mapped_product.lower())
        
        # Get product-specific returns
        product_cols = [col for col in df_raw.columns if col.startswith(f'{product_lower}_A') and '/' not in col]
        product_cols = sorted(product_cols, key=lambda x: int(x.split('_A')[1]))
        df_prices = df_raw[product_cols].copy()
        df_returns = df_prices.diff().dropna()
        is_holiday = [date.date() in holiday_dates for date in df_returns.index]
        df_returns = df_returns[~pd.Series(is_holiday, index=df_returns.index)]
        
        # Compute single-product covariance
        n_front = len(front)
        n_mid = len(mid)
        n_back = len(back)
        
        Sigma_front = compute_single_product_ewma_cov(df_returns, front, product_lower, lambda_front, ewma_init_obs)
        Sigma_mid = compute_single_product_ewma_cov(df_returns, mid, product_lower, lambda_mid, ewma_init_obs)
        Sigma_back = compute_single_product_ewma_cov(df_returns, back, product_lower, lambda_back, ewma_init_obs)
        
        Sigma_total_sp = np.zeros((n_total, n_total))
        Sigma_total_sp[:n_front, :n_front] = Sigma_front
        Sigma_total_sp[n_front:n_front+n_mid, n_front:n_front+n_mid] = Sigma_mid
        Sigma_total_sp[n_front+n_mid:, n_front+n_mid:] = Sigma_back
        
        # Build total position vector for this product
        w_total_sp = np.zeros(n_total)
        if mapped_product in delta_summary_df.columns:
            for _, row in delta_summary_df.iterrows():
                tenor = row['Tenor']
                position = row[mapped_product]
                if abs(position) > 1e-10 and tenor in contract_to_node:
                    node = contract_to_node[tenor]
                    node_idx = all_nodes.index(node)
                    w_total_sp[node_idx] = float(position)
        
        # Get strategies for this product
        product_strategies = delta_positions_df[
            delta_positions_df['Mapped_Product'] == mapped_product
        ]['Strategy'].unique()
        
        for strategy_name in product_strategies:
            w_strategy = build_strategy_position_vector(strategy_name, mapped_product, delta_positions_df)
            strategy_qty = np.sum(np.abs(w_strategy))
            
            if strategy_qty < 1e-10:
                continue
            
            # MC with independence assumption
            numerator_sp = w_strategy.T @ Sigma_total_sp @ w_total_sp
            total_var_sp = w_total_sp.T @ Sigma_total_sp @ w_total_sp
            if total_var_sp > 0:
                mc_original = 1000.0 * numerator_sp / np.sqrt(total_var_sp)
            else:
                mc_original = 0.0
            
            position_mc_results.append({
                'Strategy': strategy_name,
                'Product': mapped_product,
                'MC_original': mc_original,
                'Qty_total': strategy_qty
            })
    
    # -------------------------------------------------------------------------
    # STEP 5: Calculate MC with multi-product covariance
    # -------------------------------------------------------------------------
    for result in position_mc_results:
        strategy_name = result['Strategy']
        mapped_product = result['Product']
        
        if mapped_product not in product_indices:
            result['MC_multi_product'] = 0.0
            continue
        
        i_start, i_end = product_indices[mapped_product]
        
        w_strategy = build_strategy_position_vector(strategy_name, mapped_product, delta_positions_df)
        w_strategy_combined = np.zeros(n_combined)
        w_strategy_combined[i_start:i_end] = w_strategy
        
        numerator = w_strategy_combined.T @ Sigma_multi @ w_total_combined
        denominator = np.sqrt(w_total_combined.T @ Sigma_multi @ w_total_combined)
        
        if denominator > 1e-10:
            result['MC_multi_product'] = 1000.0 * numerator / denominator
        else:
            result['MC_multi_product'] = 0.0
    
    # -------------------------------------------------------------------------
    # STEP 6: Aggregate by Strategy
    # -------------------------------------------------------------------------
    mc_df = pd.DataFrame(position_mc_results)
    
    mc_by_strategy_multi = mc_df.groupby('Strategy').agg({
        'MC_original': 'sum',
        'MC_multi_product': 'sum',
        'Qty_total': 'sum'
    }).reset_index()
    
    mc_by_strategy_multi['Difference'] = mc_by_strategy_multi['MC_multi_product'] - mc_by_strategy_multi['MC_original']
    mc_by_strategy_multi['Pct_change'] = (
        mc_by_strategy_multi['Difference'] / mc_by_strategy_multi['MC_original'].abs() * 100
    ).replace([np.inf, -np.inf], 0.0)
    
    mc_by_strategy_multi = mc_by_strategy_multi.sort_values(
        'MC_multi_product', key=abs, ascending=False
    ).reset_index(drop=True)
    
    return mc_by_strategy_multi


print("Function 'compute_strategy_mc_table' defined successfully.")

Function 'compute_strategy_mc_table' defined successfully.


In [89]:
# =============================================================================
# FUNCTION: compute_hedge_recommendations
# =============================================================================

def compute_hedge_recommendations(
    df_raw,
    delta_positions_df,
    delta_summary_df,
    holiday_dates,
    hedge_products=None,
    top_n=20,
    front=["A01", "A02", "A03", "A04"],
    mid=["A05", "A06", "A07", "A08"],
    back=["A09", "A10", "A11", "A12", "A13", "A14", "A15"],
    lambda_front=0.97,
    lambda_mid=0.98,
    lambda_back=0.99,
    ewma_init_obs=60,
    mapped_to_data_column=None
):
    """
    Compute top hedge recommendations using multi-product covariance.
    
    Returns DataFrame with: hedge_instrument, product, beta, risk_reduction, 
                           current_risk, hedged_risk, recommended_lots
    """
    if hedge_products is None:
        hedge_products = ['HTT', 'CLBR']
    if mapped_to_data_column is None:
        mapped_to_data_column = {'HTT': 'htt', 'HOUBR': 'houbr', 'CLBR': 'clbr', 'WDF': 'wdf', 'LH': 'lh'}
    
    all_nodes = [f"A{i:02d}" for i in range(1, 16)]
    unique_tenors = delta_summary_df['Tenor'].unique()
    contract_to_node = {tenor: f"A{idx+1:02d}" for idx, tenor in enumerate(unique_tenors[:15])}
    all_products = sorted(delta_positions_df['Mapped_Product'].unique())
    
    # Build returns matrix
    products_with_data, product_returns_dict = [], {}
    for mapped_product in all_products:
        product_lower = mapped_to_data_column.get(mapped_product, mapped_product.lower())
        product_cols = [col for col in df_raw.columns if col.startswith(f'{product_lower}_A') and '/' not in col]
        if not product_cols:
            continue
        product_cols = sorted(product_cols, key=lambda x: int(x.split('_A')[1]))
        df_returns = df_raw[product_cols].diff().dropna()
        is_holiday = [date.date() in holiday_dates for date in df_returns.index]
        df_returns = df_returns[~pd.Series(is_holiday, index=df_returns.index)]
        if len(df_returns) >= ewma_init_obs:
            products_with_data.append(mapped_product)
            product_returns_dict[mapped_product] = df_returns
    
    if not products_with_data:
        raise ValueError("No products with sufficient data")
    
    common_dates = product_returns_dict[products_with_data[0]].index
    for product in products_with_data[1:]:
        common_dates = common_dates.intersection(product_returns_dict[product].index)
    
    combined_returns_list, column_order = [], []
    for product in products_with_data:
        product_lower = mapped_to_data_column.get(product, product.lower())
        product_returns = product_returns_dict[product].loc[common_dates]
        for col in sorted(product_returns.columns, key=lambda x: int(x.split('_A')[1])):
            combined_returns_list.append(product_returns[col])
            column_order.append(col)
    
    combined_returns_df = pd.DataFrame(dict(zip(column_order, combined_returns_list)))
    product_indices = {product: (i*15, (i+1)*15) for i, product in enumerate(products_with_data)}
    n_combined = len(products_with_data) * 15
    
    # Compute EWMA covariance
    returns_array = combined_returns_df.values
    n_obs, n_vars = returns_array.shape
    init_returns = returns_array[:ewma_init_obs][~np.isnan(returns_array[:ewma_init_obs]).any(axis=1)]
    cov_current = np.cov(init_returns.T) if len(init_returns) >= 10 else np.eye(n_vars) * np.var(returns_array, axis=0).mean()
    
    lambda_vec = np.zeros(n_vars)
    for product in products_with_data:
        i_start, _ = product_indices[product]
        for node_idx in range(15):
            node_code = all_nodes[node_idx]
            lambda_vec[i_start + node_idx] = lambda_front if node_code in front else lambda_mid if node_code in mid else lambda_back
    
    for t in range(ewma_init_obs, n_obs):
        r_t = returns_array[t:t+1, :]
        if not np.isnan(r_t).any():
            outer_product = np.outer(r_t[0], r_t[0])
            lambda_matrix = (np.outer(lambda_vec, np.ones(n_vars)) + np.outer(np.ones(n_vars), lambda_vec)) / 2
            cov_current = lambda_matrix * cov_current + (1 - lambda_matrix) * outer_product
    
    Sigma_multi = cov_current
    
    # Build position vector
    w_total_combined = np.zeros(n_combined)
    for product in products_with_data:
        if product in product_indices and product in delta_summary_df.columns:
            i_start, _ = product_indices[product]
            for _, row in delta_summary_df.iterrows():
                if abs(row[product]) > 1e-10 and row['Tenor'] in contract_to_node:
                    w_total_combined[i_start + all_nodes.index(contract_to_node[row['Tenor']])] = float(row[product])
    
    # Build hedge universe
    hedge_instruments = all_nodes.copy()
    for bucket in [front, mid, back]:
        hedge_instruments += [f"{bucket[i]}/{bucket[i+1]}" for i in range(len(bucket)-1)]
    if len(back) >= 4:
        hedge_instruments.append(f"{back[0]}/{back[3]}")
    if len(back) >= 6:
        hedge_instruments.append(f"{back[2]}/{back[5]}")
    
    current_risk = np.sqrt(w_total_combined.T @ Sigma_multi @ w_total_combined)
    hedge_results = []
    
    for hedge_product in hedge_products:
        if hedge_product not in product_indices:
            continue
        i_start, _ = product_indices[hedge_product]
        
        for instr in hedge_instruments:
            h_hedge = np.zeros(n_combined)
            if '/' in instr:
                parts = instr.split('/')
                if parts[0] in all_nodes and parts[1] in all_nodes:
                    h_hedge[i_start + all_nodes.index(parts[0])] = 1.0
                    h_hedge[i_start + all_nodes.index(parts[1])] = -1.0
            elif instr in all_nodes:
                h_hedge[i_start + all_nodes.index(instr)] = 1.0
            
            if np.sum(np.abs(h_hedge)) < 1e-10:
                continue
            
            h_Σ_h = h_hedge.T @ Sigma_multi @ h_hedge
            if h_Σ_h <= 0:
                continue
            
            w_Σ_h = w_total_combined.T @ Sigma_multi @ h_hedge
            beta = -w_Σ_h / h_Σ_h
            w_hedged = w_total_combined + beta * h_hedge
            hedged_risk = np.sqrt(w_hedged.T @ Sigma_multi @ w_hedged)
            risk_reduction = (current_risk - hedged_risk) / current_risk if current_risk > 0 else 0.0
            
            hedge_results.append({
                'hedge_instrument': instr, 'product': hedge_product, 'beta': beta,
                'risk_reduction': risk_reduction, 'current_risk': current_risk,
                'hedged_risk': hedged_risk, 'recommended_lots': beta
            })
    
    hedge_df = pd.DataFrame(hedge_results)
    if len(hedge_df) > 0:
        hedge_df = hedge_df.sort_values('risk_reduction', ascending=False).head(top_n).reset_index(drop=True)
    return hedge_df


print("Function 'compute_hedge_recommendations' defined successfully.")

Function 'compute_hedge_recommendations' defined successfully.


In [90]:
# =============================================================================
# Test: compute_strategy_mc_table
# =============================================================================
print("Testing compute_strategy_mc_table...")
mc_strategy_table = compute_strategy_mc_table(df_raw, delta_positions_df, delta_summary_df, holiday_dates)
print(mc_strategy_table.to_string(index=False))

Testing compute_strategy_mc_table...
   Strategy   MC_original  MC_multi_product  Qty_total    Difference  Pct_change
  HTT_Rolls 169694.483195      95506.005492  8900.0000 -74188.477704  -43.718851
HOUBR_rolls  71660.240209      26772.945462  5000.0000 -44887.294747  -62.639051
    HTT_Mid  16519.098127      15314.743853   574.0000  -1204.354273   -7.290678
 HOUBR_Back   8404.973125      13569.855723   756.9996   5164.882598   61.450317
HOUBR_Front   3308.351864       3398.469168   600.0000     90.117304    2.723933
  HTT_Front  -2971.160649      -2046.601924   100.0000    924.558725   31.117763
   HTT_Back   5025.840982       1380.447689   975.0000  -3645.393292  -72.533001
    Freight  28025.120437        611.661863   145.0000 -27413.458575  -97.817451


In [91]:
# =============================================================================
# Test: compute_hedge_recommendations
# =============================================================================
print("Testing compute_hedge_recommendations...")
top_recommendations = compute_hedge_recommendations(df_raw, delta_positions_df, delta_summary_df, holiday_dates, top_n=10)
print(top_recommendations.to_string(index=False))

Testing compute_hedge_recommendations...
hedge_instrument product        beta  risk_reduction  current_risk  hedged_risk  recommended_lots
             A03     HTT 3135.550477        0.322840    154.507527   104.626294       3135.550477
             A02     HTT 2244.716097        0.253719    154.507527   115.306011       2244.716097
         A03/A04     HTT 3441.714294        0.215224    154.507527   121.253730       3441.714294
             A01     HTT 1178.780939        0.081382    154.507527   141.933397       1178.780939
             A08    CLBR -935.206244        0.075998    154.507527   142.765219       -935.206244
             A07    CLBR -897.715271        0.070331    154.507527   143.640926       -897.715271
             A09    CLBR -916.414391        0.068735    154.507527   143.887507       -916.414391
             A10    CLBR -899.544758        0.064816    154.507527   144.492991       -899.544758
             A15    CLBR -895.533532        0.061030    154.507527   145.0779

In [92]:
!git push

Username for 'https://github.com': ^C
